In [4]:
import re
from selenium import webdriver

import time

from bs4 import BeautifulSoup
import requests

import pandas as pd

# Scraping the Oyez Project Website

## Selenium + BeautifulSoup

In [13]:
a = 1

driver = webdriver.Chrome()
oyez_list = []

while a < 73:
    oyez_case_dict = {}
    
    driver.get("https://www.oyez.org/cases/2016")
    
    time.sleep(3)
    granted = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/aside/div[1]')
    oyez_case_dict['granted'] = granted.text.replace('GRANTED', '').strip()
    
    argued = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/aside/div[2]')
    oyez_case_dict['argued'] = argued.text.replace('ARGUED', '').strip()
    
    decided = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/aside/div[3]')
    oyez_case_dict['decided'] = decided.text.replace('DECIDED', '').strip()
    
    summary = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/div')
    oyez_case_dict['summary'] = summary.text.strip()
    
    title = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/h2/a')
    title.click()
    
    time.sleep(3)
    
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    cells = doc.find_all('div', attrs = {'class': 'row'})
    
    location = cells[0].find('div', attrs = {'class': 'cell ng-scope'})
    if location:
        location = location.a['iframe-url']
        oyez_case_dict['location'] = re.findall(r'q=(.*)', location)[0]
    
    time.sleep(1)
    petitioner = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/aside/div[1]/div[1]')
    oyez_case_dict['petitioner'] = petitioner.text.replace('PETITIONER', '').strip()
    
    respondent = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/aside/div[1]/div[2]')
    oyez_case_dict['respondent'] = respondent.text.replace('RESPONDENT', '').strip()
    
    docket = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/aside/div[2]/div[1]')
    oyez_case_dict['docket'] = docket.text.replace('DOCKET NO.', '').strip()
    
    
    time.sleep(1)
    fact = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/section[1]/div')
    oyez_case_dict['descriptions'] = fact.text.strip()

    time.sleep(1)
    conclusion = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/section[1]/div')
    oyez_case_dict['conclusion'] = conclusion.text.strip()

    oyez_list.append(oyez_case_dict)
    
    print(a)
    a += 1
    
    time.sleep(1)

driver.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


In [479]:
b = 1

oyez_list2 = []
driver = webdriver.Chrome()

while b < 73:

    oyez_case_dic = {}

    driver.get("https://www.oyez.org/cases/2016")
    
    time.sleep(3)

    title = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(b) +']/h2/a')
    title.click()

    time.sleep(3)

    doc = BeautifulSoup(driver.page_source, 'html.parser')
        
    maj_opin = doc.find('li', attrs = {'ng-if':'document.justia_opinion_url'})
    if maj_opin:
        oyez_case_dic['docket'] = maj_opin.a['href']

    time.sleep(1)

    docket = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/aside/div[2]/div[1]')
    oyez_case_dic['docket'] = docket.text.replace('DOCKET NO.', '').strip()


    time.sleep(1)
    
    try:
        vote = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/section[3]/div[1]/div[3]/ul/li/figure/figcaption/h3')
        oyez_case_dic['vote'] = vote.text.strip()
    except:
        pass

    try:
        s_conclusion = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/section[3]/div[1]/div[3]/ul/li/figure/figcaption/p')
        oyez_case_dic['s_conclusion'] = s_conclusion.text.strip()
    except:
        pass
    
    oyez_list2.append(oyez_case_dic)
    
    time.sleep(1)
    print(b)
    b += 1
    
driver.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


In [468]:
info_df2 = pd.DataFrame(oyez_list2)
info_df2

docket                                       s_conclusion  \
0     16-74  The “church plan” exemption of the Employee Re...   
1   15-1111  The “aggrieved persons” language of the Fair H...   
2   15-8544  No. The Court’s holding in Johnson v. US does ...   
3    15-680  The District Court employed an incorrect legal...   
4    16-405  The Federal Employers’ Liability Act (FELA) do...   
5   15-9173                                                NaN   
6    15-537  The issue-preclusion component of the Double J...   
7    16-466  When the plaintiffs’ claims would have been ex...   
8   15-8049  The Court held that the Fifth Circuit applied ...   
9    16-373  The filing of a putative class action lawsuit ...   
10  15-1262  The district court did not error in determinin...   
11   16-369  The U.S. Court of Appeals for the Ninth Circui...   
12   16-149  The Federal Employees Health Benefits Act (FEH...   
13   15-649  Bankruptcy courts may not approve structured d...   
14  16-6219                                                NaN   
15  15-9260  Section 924(c) does not prevent a sentencing c...   
16   15-827  To meet its substantive obligation under the I...   
17    16-54  The crime of “unlawful sexual intercourse with...   
18  15-1391  The New York law was not unconstitutionally va...   
19   15-497  The Individuals with Disabilities Education Ac...   
20   16-273                                                NaN   
21  15-1406  When a federal court exercises its inherent au...   
22   16-349  A party that purchases a debt and attempts to ...   
23   15-118                                                NaN   
24   16-142  The federal forfeiture statute does not requir...   
25  15-1031  The Uniformed Services Former Spouses’ Protect...   
26  15-1189  A patentee’s decision to sell an item exhausts...   
27   15-486                                                NaN   
28  16-1116                                                NaN   
29    16-32  The Federal Arbitration Act (FAA) preempts Ken...   
..      ...                                                ...   
42   16-348  The Fair Debt Collection Practices Act (FDCPA)...   
43   15-797  The use of outdated medical standards regardin...   
44   15-214  A case in which the Court determined that, in ...   
45  15-1251  The limitations on nominees to vacant position...   
46  15-1256  The Colorado Exoneration Act, which requires e...   
47                                                         NaN   
48  15-1194  The North Carolina law prohibiting registered ...   
49   16-992  A state law that precludes same-sex couples fr...   
50   15-606  Where a juror makes a clear statement indicati...   
51   16-399  A Merit Systems Protection Board decision to d...   
52                                                         NaN   
53   15-628  The jury could infer that the tipper here pers...   
54   15-777  In the case of a multicomponent product, the r...   
55  15-1039  The Biologics Price Competition and Innovation...   
56   15-927  Laches cannot be invoked as a defense against ...   
57  15-1191  The statutory distinction between the physical...   
58  15-5991  To convict a defendant of defrauding a financi...   
59   15-866  A feature incorporated into the design of a us...   
60   15-513  A seal violation does not mandate dismissal of...   
61   16-341  The subsection of the general venue statute th...   
62   16-605  An intervenor to a lawsuit must meet the requi...   
63   15-577  The exclusion of churches from an otherwise ne...   
64  15-1503  The precedent of Brady v. Maryland does not re...   
65   15-423  The “frivolous or insubstantial” standard is n...   
66                                                         NaN   
67   15-961                                                NaN   
68   16-254  The Hague Service Convention does not prohibit...   
69   16-240  In the context of the violation of a right to ...   
70    16-67  An officer’s failure to shout a 

In [480]:
info_df = pd.DataFrame(oyez_list)
info_df

argued                                         conclusion  \
0   Mar 27, 2017  The Employee Retirement Income Security Act of...   
1    Nov 8, 2016  The city of Miami sued Bank of America Corpora...   
2   Nov 28, 2016  On April 11, 2007, Travis Beckles was arrested...   
3    Dec 5, 2016  Based on the 2010 Census, the Virginia General...   
4   Apr 25, 2017  In March 2011, Robert Nelson was employed by B...   
5     Not argued  In 1987, the U.S. Supreme Court decided in Boo...   
6    Oct 4, 2016  In May 2005, Juan Bravo-Fernandez, the preside...   
7   Apr 25, 2017  Bristol-Myers Squibb Co. (BMS) is a global pha...   
8    Oct 5, 2016  In July 1995, Duane Edward Buck was arrested f...   
9   Apr 17, 2017  Before it went bankrupt in 2008, Lehman Brothe...   
10   Dec 5, 2016  After the 2010 Census, pursuant to the state C...   
11  Mar 22, 2017  In October 2010, Deputies Christopher Conley a...   
12   Mar 1, 2017  Jodie Nevils was a federal employee with a Cov...   
13   Dec 7, 2016  Jevic Transportation, Inc. was a trucking comp...   
14  Apr 24, 2017  In February 2009, Erick Daniel Davila was foun...   
15  Feb 28, 2017  Levon Dean, Jr. and Jamal Dean were brothers w...   
16  Jan 11, 2017  Endrew F. is an autistic fifth grade student w...   
17  Feb 27, 2017  Juan Esquivel-Quintana was admitted to the Uni...   
18  Jan 10, 2017  The New York General Business Law prohibits su...   
19  Oct 31, 2016  The Frys’ daughter, E.F., was born with cerebr...   
20       Pending  During the 2013-14 school year, G.G., a transg...   
21  Jan 10, 2017  In 2003, Leroy, Donna, Barry, and Suzanne Haeg...   
22  Apr 18, 2017  The petitioners are a group of individuals who...   
23  Feb 21, 2017  On June 7, 2010, Sergio Adrian Hernandez Guere...   
24  Mar 29, 2017  Terry Honeycutt was a salaried employee who wo...   
25  Mar 20, 2017  John Howell, a veteran, and his ex-wife Sandra...   
26  Mar 21, 2017  Lexmark International, Inc. (Lexmark) owns man...   
27       Pending  In Texas, people under the age of 25 must subm...   
28    Not argued  In 1985, Percy Hutton allegedly lured two frie...   
29  Feb 22, 2017  Olive Clark and Joe Wellner were both resident...   
..           ...                                                ...   
42  Jan 17, 2017  In 2014, Aleida Johnson filed for bankruptcy i...   
43  Nov 29, 2016  In 1980, Bobby James Moore was convicted of ca...   
44  Mar 20, 2017  The parents of Joseph P. Murr and his siblings...   
45   Nov 7, 2016  In 1998, Congress enacted the Federal Vacancie...   
46   Jan 9, 2017  The state of Colorado, like most states, impos...   
47    Not argued                                                      
48  Feb 27, 2017  Lester Packingham was convicted of taking “ind...   
49    Not argued  Leah and Jana Jacobs, and Terrah and Marisa Pa...   
50  Oct 11, 2016  Miguel Angel Pena-Rodriguez was convicted of u...   
51  Apr 17, 2017  In the mid-2000s, Anthony Perry began to devel...   
52    Not argued                                                      
53   Oct 5, 2016  Maher Kara joined Citigroup’s healthcare inves...   
54  Oct 11, 2016  In April 2011, Apple Inc. (Apple) sued Samsung...   
55  Apr 26, 2017  The Biologics Price Competition and Innovation...   
56   Nov 1, 2016  SCA Hygiene Products Aktiebolag (SCA) produces...   
57   Nov 9, 2016  Luis Ramon Morales-Santana was born in 1962 in...   
58   Oct 4, 2016  Stanley Hsu, a Taiwanese business man, opened ...   
59  Oct 31, 2016  Varsity Brands, Inc. (Varsity) designs and man...   
60   Nov 1, 2016  Many homeowners in the Gulf Coast area had two...   
61  Mar 27, 2017  TC Heartland LLC (Heartland) is a company orga...   
62  Apr 17, 2017  Land developer Steve Sherman sued the Town of ...   
63  Apr 19, 2017  Trinity Lutheran Church of Columbia, Inc. (Tri...   
64  Mar 29, 2017  In 1984, the body of Catherine Fuller was disc...   
65   Nov 2, 2016  The Foreign Sovereign Immunities Act protects ...   
66    Not argued                  

# Scraping the Supreme Court Website

In [41]:
driver = webdriver.Chrome()
driver.get("https://www.supremecourt.gov/oral_arguments/argument_transcript.aspx")
doc = BeautifulSoup(driver.page_source, 'html.parser')
caseno_list = doc.find_all('a')
case_list = []
case_dict = {}

caseno_list = caseno_list[86:150]

caseno_flist = []
caseno_link = []
file_name_list = []
for case in caseno_list:
    tran_link = case['href']
    caseno_link.append(tran_link)
    regex10 = r'[/]([0-9]{2}-[0-9]+_[0-9a-z]+).pdf$'
    file_name = re.findall(regex10, tran_link)
    for file in file_name:
        file_name_list.append(file)
    case = case.text.strip().replace('.','')
    caseno_flist.append(case)
    
name_list = doc.find_all('span')
name_list = name_list[18:82]
name_flist = []
for name in name_list:
    name = name.text.strip()
    name_flist.append(name)
    
    
# date_list = doc.find_all('tr')
# date_list = date_list[3:73]
# date_flist = []
# for date_cell in date_list:
#     date = date_cell.find_all('td')[1].text.strip()
#     if date != 'Date Argued':
#         date_flist.append(date)
        
driver.close()

df = pd.DataFrame({
    'docket': caseno_flist,
    'caseName': name_flist,
    #'date argued': date_flist,
    'transcriptLink': caseno_link,
    'fileName': file_name_list
})

In [457]:
df

caseName   docket      fileName  \
0               Perry v. Merit Systems Protection Bd.   16-399   16-399_3f14   
1              Town of Chester v. Laroe Estates, Inc.   16-605   16-605_2dp3   
2   California Public Employees' Retirement System...   16-373   16-373_4e46   
3                                       Kokesh v. SEC   16-529   16-529_21p3   
4               Henson v. Santander Consumer USA Inc.   16-349   16-349_e29g   
5   Trinity Lutheran Church of Columbia, Inc. v. C...   15-577   15-577_l64n   
6                             Weaver v. Massachusetts   16-240   16-240_nkp1   
7                                  McWilliams v. Dunn  16-5294  16-5294_g314   
8                                     Davila v. Davis  16-6219  16-6219_7mio   
9   Bristol-Myers Squibb Co. v. Superior Court of ...   16-466   16-466_4g15   
10                             BNSF R. Co. v. Tyrrell   16-405   16-405_9olb   
11                          Sandoz Inc. v. Amgen Inc.  15-1039  15-1039_bqm1   
12                         Maslenjak v. United States   16-309   16-309_b97c   
13                                  Murr v. Wisconsin   15-214   15-214_l6hn   
14                                   Howell v. Howell  15-1031  15-1031_6647   
15                           Microsoft Corp. v. Baker   15-457   15-457_gfbh   
16  Impressions Products, Inc. v. Lexmark Int'l, Inc.  15-1189  15-1189_6468   
17                    County of Los Angeles v. Mendez   16-369   16-369_8nka   
18                        Water Splash, Inc. v. Menon   16-254   16-254_7lio   
19          Advocate Health Care Network v. Stapleton    16-74    16-74_p8k0   
20   TC Heartland LLC v. Kraft Foods Group Brands LLC   16-341   16-341_8njq   
21                               Lee v. United States   16-327   16-327_d18e   
22                            Turner v. United States  15-1503  15-1503_3f14   
23                         Honeycutt v. United States   16-142   16-142_4gc5   
24                                  Hernandez v. Mesa   15-118   15-118_3e04   
25                                 McLane Co. v. EEOC  15-1248  15-1248_2dq3   
26              Kindred Nursing Centers Ltd. v. Clark    16-32    16-32_mlho   
27                       Packingham v. North Carolina  15-1194  15-1194_0861   
28                      Esquivel-Quintana v. Sessions    16-54    16-54_7l48   
29                              Dean v. United States  15-9260  15-9260_bq7c   
..                                                ...      ...           ...   
34               Goodyear Tire & Rubber Co. v. Haeger  15-1406  15-1406_d1of   
35      Endrew F. v. Douglas County School Dist. RE-1   15-827   15-827_gfbh   
36                                    Lynch v. Dimaya  15-1498  15-1498_m647   
37                    Midland Funding, LLC v. Johnson   16-348   16-348_2cp3   
38                                         Lee v. Tam  15-1293  15-1293_o7jp   
39                                   Ziglar v. Abbasi  15-1358  15-1358_7648   
40                           Beckles v. United States  15-8544  15-8544_c1o2   
41                                     Moore v. Texas   15-797   15-797_f2q3   
42                              Jennings v. Rodriguez  15-1204  15-1204_k536   
43    Bethune-Hill v. Virginia State Bd. of Elections   15-680   15-680_n648   
44                                  McCrory v. Harris  15-1262  15-1262_l537   
45           Life Technologies Corp. v. Promega Corp.  14-1538  14-1538_j4ek   
46                   Czyzewski v. Jevic Holding Corp.   15-649   15-649_l5gm   
47                  Fry v. Napoleon Community Schools   15-497   15-497_4g15   
48     Star Athletica, L.L.C. v. Varsity Brands, Inc.   15-866   15-866_j426   
49  State Farm Fire & Casualty Co. v. United State...   15-513   15-513_k5fm   
50  SCA Hygiene Products Aktiebolag v. First Quali...   15-927   15-927_6j37   
51  Bolivarian Republic of Venezuela v. Helmerich ...   15-423   15-423_pnk0   
52                           NLRB v. SW Gener

In [481]:
f_df = df.merge(info_df, left_on="docket", right_on="docket")
f_df

caseName   docket      fileName  \
0               Perry v. Merit Systems Protection Bd.   16-399   16-399_3f14   
1              Town of Chester v. Laroe Estates, Inc.   16-605   16-605_2dp3   
2   California Public Employees' Retirement System...   16-373   16-373_4e46   
3                                       Kokesh v. SEC   16-529   16-529_21p3   
4               Henson v. Santander Consumer USA Inc.   16-349   16-349_e29g   
5   Trinity Lutheran Church of Columbia, Inc. v. C...   15-577   15-577_l64n   
6                             Weaver v. Massachusetts   16-240   16-240_nkp1   
7                                  McWilliams v. Dunn  16-5294  16-5294_g314   
8                                     Davila v. Davis  16-6219  16-6219_7mio   
9   Bristol-Myers Squibb Co. v. Superior Court of ...   16-466   16-466_4g15   
10                             BNSF R. Co. v. Tyrrell   16-405   16-405_9olb   
11                          Sandoz Inc. v. Amgen Inc.  15-1039  15-1039_bqm1   
12                         Maslenjak v. United States   16-309   16-309_b97c   
13                                  Murr v. Wisconsin   15-214   15-214_l6hn   
14                                   Howell v. Howell  15-1031  15-1031_6647   
15                           Microsoft Corp. v. Baker   15-457   15-457_gfbh   
16  Impressions Products, Inc. v. Lexmark Int'l, Inc.  15-1189  15-1189_6468   
17                    County of Los Angeles v. Mendez   16-369   16-369_8nka   
18                        Water Splash, Inc. v. Menon   16-254   16-254_7lio   
19          Advocate Health Care Network v. Stapleton    16-74    16-74_p8k0   
20   TC Heartland LLC v. Kraft Foods Group Brands LLC   16-341   16-341_8njq   
21                               Lee v. United States   16-327   16-327_d18e   
22                            Turner v. United States  15-1503  15-1503_3f14   
23                         Honeycutt v. United States   16-142   16-142_4gc5   
24                                  Hernandez v. Mesa   15-118   15-118_3e04   
25                                 McLane Co. v. EEOC  15-1248  15-1248_2dq3   
26              Kindred Nursing Centers Ltd. v. Clark    16-32    16-32_mlho   
27                       Packingham v. North Carolina  15-1194  15-1194_0861   
28                      Esquivel-Quintana v. Sessions    16-54    16-54_7l48   
29                              Dean v. United States  15-9260  15-9260_bq7c   
..                                                ...      ...           ...   
32                                    Lewis v. Clarke  15-1500  15-1500_5g68   
33            Expressions Hair Design v. Schneiderman  15-1391  15-1391_5315   
34               Goodyear Tire & Rubber Co. v. Haeger  15-1406  15-1406_d1of   
35      Endrew F. v. Douglas County School Dist. RE-1   15-827   15-827_gfbh   
36                    Midland Funding, LLC v. Johnson   16-348   16-348_2cp3   
37                                         Lee v. Tam  15-1293  15-1293_o7jp   
38                                   Ziglar v. Abbasi  15-1358  15-1358_7648   
39                           Beckles v. United States  15-8544  15-8544_c1o2   
40                                     Moore v. Texas   15-797   15-797_f2q3   
41    Bethune-Hill v. Virginia State Bd. of Elections   15-680   15-680_n648   
42                                  McCrory v. Harris  15-1262  15-1262_l537   
43           Life Technologies Corp. v. Promega Corp.  14-1538  14-1538_j4ek   
44                   Czyzewski v. Jevic Holding Corp.   15-649   15-649_l5gm   
45                  Fry v. Napoleon Community Schools   15-497   15-497_4g15   
46     Star Athletica, L.L.C. v. Varsity Brands, Inc.   15-866   15-866_j426   
47  State Farm Fire & Casualty Co. v. United State...   15-513   15-513_k5fm   
48  SCA Hygiene Products Aktiebolag v. First Quali...   15-927   15-927_6j37   
49  Bolivarian Republic of Venezuela v. Helmerich ...   15-423   15-423_pnk0   
50                           NLRB v. SW Gener

In [231]:
import geopandas as gpd
from shapely.geometry import Point

%matplotlib inline

# Scraping the oral arugment transcripts

## regular expressions

In [10]:
def scraping(sample_transcript):
    trans_list = []
    
    
    my_reg = r"Alderson Reporting Company[^S]*Subject to Final Review"
    sample_transcript = re.sub(my_reg," ",sample_transcript)
    
    #sample_transcript = sample_transcript.replace('Alderson Reporting Company', '')
    #sample_transcript = sample_transcript.replace('Official - Subject to Final Review', '')

    regex = r"\b[12]{0,1}[0-9]?\b"
    sample_transcript = re.sub(regex, '', sample_transcript)

    chop_top = sample_transcript.split("PROCEEDINGS")
    sample_transcript = chop_top[1]

    regex2 = r"(submitted.[)])"
    chop_end = re.split(regex2,sample_transcript)
    sample_transcript = chop_end[0] + chop_end[1]

    regex3 = r'([A-Z]+[.]{0,1}[ ]{0,1}[A-Z]+\W{0,2}[A-Z]+):'

    transcript_list = re.split(regex3,sample_transcript)
    transcript_list

    transcript_list = transcript_list[1:]

    d = 0
    speaker_list = []
    word_list = []
    
    
    
    for speaker in transcript_list:
        if d % 2 == 0:
            speaker_list.append(speaker.strip())
        else:
            word_list.append(speaker.strip())
        d = d + 1
    
    for item1,item2 in zip(speaker_list, word_list):
        trans_dict = {}
        trans_dict["speaker"] = item1
        trans_dict["word"] = item2
        trans_list.append(trans_dict)
        
    return trans_list

In [11]:
all_trans_list = []
z = 0

for file, case in zip(file_name_list, caseno_flist):
    f = open('/Users/Shiying/Documents/ColumbiaLede/data_database/supreme_court/pdfs/' + file + '.txt', 'rb')
    
    try:
        #print('/Users/Shiying/Documents/ColumbiaLede/data_database/supreme_court/pdfs/' + file + '.txt')
        this_transcript = f.read().decode('utf8', 'ignore')
        trans_list = scraping(this_transcript)
        for item in trans_list:
            item['docket'] = case
        
        all_trans_list.extend(trans_list)
        
    except:
        pass
    
df2 = pd.DataFrame(all_trans_list)

In [12]:
df2

docket                speaker  \
0       16-399  CHIEF JUSTICE ROBERTS   
1       16-399             MR. LANDAU   
2       16-399      JUSTICE SOTOMAYOR   
3       16-399        JUSTICE KENNEDY   
4       16-399             MR. LANDAU   
5       16-399        JUSTICE KENNEDY   
6       16-399       JUSTICE GINSBURG   
7       16-399             MR. LANDAU   
8       16-399       JUSTICE GINSBURG   
9       16-399             MR. LANDAU   
10      16-399  CHIEF JUSTICE ROBERTS   
11      16-399             MR. LANDAU   
12      16-399  CHIEF JUSTICE ROBERTS   
13      16-399             MR. LANDAU   
14      16-399       JUSTICE GINSBURG   
15      16-399             MR. LANDAU   
16      16-399  CHIEF JUSTICE ROBERTS   
17      16-399             MR. LANDAU   
18      16-399  CHIEF JUSTICE ROBERTS   
19      16-399             MR. LANDAU   
20      16-399  CHIEF JUSTICE ROBERTS   
21      16-399             MR. LANDAU   
22      16-399          JUSTICE ALITO   
23      16-399             MR. LANDAU   
24      16-399          JUSTICE KAGAN   
25      16-399             MR. LANDAU   
26      16-399          JUSTICE KAGAN   
27      16-399             MR. LANDAU   
28      16-399          JUSTICE KAGAN   
29      16-399             MR. LANDAU   
...        ...                    ...   
13536  15-7250              MR. KEDEM   
13537  15-7250          JUSTICE KAGAN   
13538  15-7250              MR. KEDEM   
13539  15-7250          JUSTICE KAGAN   
13540  15-7250              MR. KEDEM   
13541  15-7250          JUSTICE KAGAN   
13542  15-7250              MR. KEDEM   
13543  15-7250          JUSTICE KAGAN   
13544  15-7250              MR. KEDEM   
13545  15-7250          JUSTICE KAGAN   
13546  15-7250              MR. KEDEM   
13547  15-7250          JUSTICE KAGAN   
13548  15-7250              MR. KEDEM   
13549  15-7250       JUSTICE GINSBURG   
13550  15-7250              MR. KEDEM   
13551  15-7250       JUSTICE GINSBURG   
13552  15-7250              MR. KEDEM   
13553  15-7250  CHIEF JUSTICE ROBERTS   
13554  15-7250           MR. RASHKIND   
13555  15-7250      JUSTICE SOTOMAYOR   
13556  15-7250           MR. RASHKIND   
13557  15-7250  CHIEF JUSTICE ROBERTS   
13558  15-7250           MR. RASHKIND   
13559  15-7250  CHIEF JUSTICE ROBERTS   
13560  15-7250           MR. RASHKIND   
13561  15-7250  CHIEF JUSTICE ROBERTS   
13562  15-7250           MR. RASHKIND   
13563  15-7250  CHIEF JUSTICE ROBERTS   
13564  15-7250           MR. RASHKIND   
13565  15-7250  CHIEF JUSTICE ROBERTS   

                                                    word  
0      We'll hear argument  first this morning in Cas...  
1      Thank you, Mr. Chief Justice,  and may it plea...  
2                                           I'm sorry --  
3      Just -- just as an\n\n introductory question, ...  
4      I don't think they will be able\n\n to, Your H...  
5                                           Certainly --  
6      But the district court\n\n couldn't resolve th...  
7      Well, you raise an interesting\n\n question, Y...  
8      The government -- the\n\n government takes the...  
9      It could take that position,\n\n \n  Your Hono...  
10     Can I -- you could  go to the district court r...  
11     Absolutely. And that's one of  the problems wi...  
12     Well, I think -- I\n\n think you're right that...  
13                       Yes, but you certainly don't --  
14     You -- you would have to\n\n exhaust before yo...  
15     Correct, Your Honor. No, I\n\n think that's co...  
16     But that -- that --\n\n that takes place -- I ...  
17                                                  Yes.  
18     -- and then you can\n\n go either to the distr...  
19                                   Correct. You can --  
20     Now, before you can  do that, you have to exha...  
21     No. The -- the -- filing your  complaint in th...  
22     If you had chosen to go  directly to the distr...  
23     Well, Your Honor, just to be\n\n clear, yo

In [49]:
df3 = df2[df2['speaker'].str.contains('JUSTICE')]

In [258]:
speak_lines = df3.sort_values(by='speaker')

docket                speaker  \
0       16-399  CHIEF JUSTICE ROBERTS   
5747   15-1248  CHIEF JUSTICE ROBERTS   
5741   15-1248  CHIEF JUSTICE ROBERTS   
5739   15-1248  CHIEF JUSTICE ROBERTS   
5735   15-1248  CHIEF JUSTICE ROBERTS   
5729   15-1248  CHIEF JUSTICE ROBERTS   
5727   15-1248  CHIEF JUSTICE ROBERTS   
5709   15-1248  CHIEF JUSTICE ROBERTS   
5649   15-1248  CHIEF JUSTICE ROBERTS   
5648    15-118  CHIEF JUSTICE ROBERTS   
5646    15-118  CHIEF JUSTICE ROBERTS   
5565    15-118  CHIEF JUSTICE ROBERTS   
5559    15-118  CHIEF JUSTICE ROBERTS   
5554    15-118  CHIEF JUSTICE ROBERTS   
5512    15-118  CHIEF JUSTICE ROBERTS   
5786   15-1248  CHIEF JUSTICE ROBERTS   
5472    15-118  CHIEF JUSTICE ROBERTS   
5468    15-118  CHIEF JUSTICE ROBERTS   
5466    15-118  CHIEF JUSTICE ROBERTS   
5464    15-118  CHIEF JUSTICE ROBERTS   
5462    15-118  CHIEF JUSTICE ROBERTS   
5452    15-118  CHIEF JUSTICE ROBERTS   
5422    15-118  CHIEF JUSTICE ROBERTS   
5416    15-118  CHIEF JUSTICE ROBERTS   
5414    15-118  CHIEF JUSTICE ROBERTS   
5412    15-118  CHIEF JUSTICE ROBERTS   
5411    15-118  CHIEF JUSTICE ROBERTS   
5407    15-118  CHIEF JUSTICE ROBERTS   
5406    16-142  CHIEF JUSTICE ROBERTS   
5404    16-142  CHIEF JUSTICE ROBERTS   
...        ...                    ...   
4990    16-327      JUSTICE SOTOMAYOR   
4992    16-327      JUSTICE SOTOMAYOR   
4994    16-327      JUSTICE SOTOMAYOR   
3757    15-457      JUSTICE SOTOMAYOR   
3755    15-457      JUSTICE SOTOMAYOR   
5008    16-327      JUSTICE SOTOMAYOR   
3786    15-457      JUSTICE SOTOMAYOR   
9162   15-1204      JUSTICE SOTOMAYOR   
11567  15-1111      JUSTICE SOTOMAYOR   
9224   15-1204      JUSTICE SOTOMAYOR   
11610  14-1055      JUSTICE SOTOMAYOR   
11608  14-1055      JUSTICE SOTOMAYOR   
11606  14-1055      JUSTICE SOTOMAYOR   
9164   15-1204      JUSTICE SOTOMAYOR   
2726    16-309      JUSTICE SOTOMAYOR   
2728    16-309      JUSTICE SOTOMAYOR   
9166   15-1204      JUSTICE SOTOMAYOR   
9168   15-1204      JUSTICE SOTOMAYOR   
9226   15-1204      JUSTICE SOTOMAYOR   
9170   15-1204      JUSTICE SOTOMAYOR   
3832    15-457      JUSTICE SOTOMAYOR   
3828    15-457      JUSTICE SOTOMAYOR   
3826    15-457      JUSTICE SOTOMAYOR   
4977    16-327      JUSTICE SOTOMAYOR   
11583  15-1111      JUSTICE SOTOMAYOR   
9218   15-1204      JUSTICE SOTOMAYOR   
9220   15-1204      JUSTICE SOTOMAYOR   
9222   15-1204      JUSTICE SOTOMAYOR   
9172   15-1204      JUSTICE SOTOMAYOR   
882     16-529      JUSTICE SOTOMAYOR   

                                                    word  
0      We'll hear argument  first this morning in Cas...  
5747   What about the\n\n burden on the other party? ...  
5741   Well, if the line  depends on relevance, it's ...  
5739   I'm sorry. Why  isn't that telling them they c...  
5735   Thank you, counsel.\n\n Mr. Kinnaird.\n\n ORAL...  
5729   Well, relevance\n\n determination is something...  
5727   As you say, Counsel,\n\n there is a bit of ten...  
5709   Thank you, counsel.  Ms. Kovner.  ORAL ARGUMEN...  
5649   We'll hear argument  next this morning in Case...  
5648   Thank you, counsel.  The case is submitted.  (...  
5646   Thank you, counsel.\n\n And, Mr. Hilliard, you...  
5565   Thank you, Counsel.  Mr. Kneedler.  ORAL ARGUM...  
5559   Could I ask you\n\n briefly, because we haven'...  
5554                                    Justice Kennedy.  
5512   Thank you, counsel.\n\n Mr. Ortega.\n\n ORAL A...  
5786   Thank you,\n\n Mr. Kinnaird.\n\n \n42  Ms. Ho,...  
5472   The question is\n\n whether the government of ...  
5468   Well, I assume\n\n that's true of a lot of bor...  
5466   It's because the\n \n\n injury occurs in a dif...  
5464   Well, but just to  stop you there. So is it an...  
5462   Well, but the one  obvious difference with Bou...  
5452   Well, your rule  would be the same if this was...  
5422   Well, I understand\n\n that, but I'm trying to...  
5416   Well, you can  imagine a situation that is

In [353]:
lines_out = speak_lines.groupby(['docket','speaker'])['word'].apply(lambda x: "<p>%s</p>" % '</p><p> '.join(x)).reset_index(name='lines')
lines_out

docket                speaker  \
0    14-1055  CHIEF JUSTICE ROBERTS   
1    14-1055         JUSTICE BREYER   
2    14-1055       JUSTICE GINSBURG   
3    14-1055          JUSTICE KAGAN   
4    14-1055        JUSTICE KENNEDY   
5    14-1055      JUSTICE SOTOMAYOR   
6    14-1538  CHIEF JUSTICE ROBERTS   
7    14-1538          JUSTICE ALITO   
8    14-1538         JUSTICE BREYER   
9    14-1538       JUSTICE GINSBURG   
10   14-1538          JUSTICE KAGAN   
11   14-1538        JUSTICE KENNEDY   
12   14-1538      JUSTICE SOTOMAYOR   
13   14-9496  CHIEF JUSTICE ROBERTS   
14   14-9496          JUSTICE ALITO   
15   14-9496         JUSTICE BREYER   
16   14-9496       JUSTICE GINSBURG   
17   14-9496          JUSTICE KAGAN   
18   14-9496        JUSTICE KENNEDY   
19   14-9496      JUSTICE SOTOMAYOR   
20   15-1031  CHIEF JUSTICE ROBERTS   
21   15-1031          JUSTICE ALITO   
22   15-1031         JUSTICE BREYER   
23   15-1031       JUSTICE GINSBURG   
24   15-1031          JUSTICE KAGAN   
25   15-1031        JUSTICE KENNEDY   
26   15-1031      JUSTICE SOTOMAYOR   
27   15-1039  CHIEF JUSTICE ROBERTS   
28   15-1039         JUSTICE BREYER   
29   15-1039       JUSTICE GINSBURG   
..       ...                    ...   
416    16-54  CHIEF JUSTICE ROBERTS   
417    16-54          JUSTICE ALITO   
418    16-54         JUSTICE BREYER   
419    16-54       JUSTICE GINSBURG   
420    16-54          JUSTICE KAGAN   
421    16-54        JUSTICE KENNEDY   
422    16-54      JUSTICE SOTOMAYOR   
423   16-605  CHIEF JUSTICE ROBERTS   
424   16-605          JUSTICE ALITO   
425   16-605         JUSTICE BREYER   
426   16-605       JUSTICE GINSBURG   
427   16-605        JUSTICE GORSUCH   
428   16-605          JUSTICE KAGAN   
429   16-605        JUSTICE KENNEDY   
430   16-605      JUSTICE SOTOMAYOR   
431  16-6219  CHIEF JUSTICE ROBERTS   
432  16-6219          JUSTICE ALITO   
433  16-6219         JUSTICE BREYER   
434  16-6219       JUSTICE GINSBURG   
435  16-6219        JUSTICE GORSUCH   
436  16-6219          JUSTICE KAGAN   
437  16-6219        JUSTICE KENNEDY   
438  16-6219      JUSTICE SOTOMAYOR   
439    16-74  CHIEF JUSTICE ROBERTS   
440    16-74          JUSTICE ALITO   
441    16-74         JUSTICE BREYER   
442    16-74       JUSTICE GINSBURG   
443    16-74          JUSTICE KAGAN   
444    16-74        JUSTICE KENNEDY   
445    16-74      JUSTICE SOTOMAYOR   

                                                 lines  
0    <p>Thank you, counsel.\n\n The case is submitt...  
1    <p>It comes out the other way.  And the Red Cr...  
2    <p>But Red Cross didn't have  the competence l...  
3    <p>Well, suppose it means  personal jurisdicti...  
4    <p>Don't tell us we're not\n\n working hard en...  
5    <p>Isn't that USG's\n\n position in Portsmouth...  
6    <p>Thank you, counsel.  The case is submitted....  
7    <p>Well, in order to do that,  it would be nec...  
8    <p>So they are worried about\n\n it, in other ...  
9    <p>And so you agree that  "substantial portion...  
10   <p>But then you take Justice  Alito's hypothet...  
11   <p>Under the U.K. laws, are  patents generally...  
12   <p>Let's assume the -</p><p> Sure.</p><p> It s...  
13   <p>Thank you, counsel.  The case is submitted....  
14   <p>On page 30 of your brief.\n\n 30 to 31 of y...  
15   <p>That's right. I mean, I\n\n may be missing ...  
16   <p>But it's a different  claim. One claim is f...  
17   <p>Mr. Eisenhammer, why should\n\n we even get...  
18   <p>Okay. It's terminated six\n\n months -- or ...  
19   <p>So you -- you don't care  that we don't rea...  
20   <p>Thank you, counsel.\n\n Mr. Wirken.\n\n ORA...  
21   <p>I think the question is can\n\n a State dep...  
22   <p>All right. That's the only\n\n difference. ...  
23   <p>What would be\n\n noncomplying with Federal...  
24   <p>But sorry to come back to  this, Mr. Unikow...  
25   <p>Is -- is the answer that\n\n there's a diff...  
26   <p>So could she go back to\n\n court now an

In [367]:
lines_out['line_dict'] = lines_out.apply(lambda row: {row['speaker']:row['lines']}, axis=1)
lines_out.to_csv('line_out.csv', index= False)

In [370]:
lines_out

docket                speaker  \
0    14-1055  CHIEF JUSTICE ROBERTS   
1    14-1055         JUSTICE BREYER   
2    14-1055       JUSTICE GINSBURG   
3    14-1055          JUSTICE KAGAN   
4    14-1055        JUSTICE KENNEDY   
5    14-1055      JUSTICE SOTOMAYOR   
6    14-1538  CHIEF JUSTICE ROBERTS   
7    14-1538          JUSTICE ALITO   
8    14-1538         JUSTICE BREYER   
9    14-1538       JUSTICE GINSBURG   
10   14-1538          JUSTICE KAGAN   
11   14-1538        JUSTICE KENNEDY   
12   14-1538      JUSTICE SOTOMAYOR   
13   14-9496  CHIEF JUSTICE ROBERTS   
14   14-9496          JUSTICE ALITO   
15   14-9496         JUSTICE BREYER   
16   14-9496       JUSTICE GINSBURG   
17   14-9496          JUSTICE KAGAN   
18   14-9496        JUSTICE KENNEDY   
19   14-9496      JUSTICE SOTOMAYOR   
20   15-1031  CHIEF JUSTICE ROBERTS   
21   15-1031          JUSTICE ALITO   
22   15-1031         JUSTICE BREYER   
23   15-1031       JUSTICE GINSBURG   
24   15-1031          JUSTICE KAGAN   
25   15-1031        JUSTICE KENNEDY   
26   15-1031      JUSTICE SOTOMAYOR   
27   15-1039  CHIEF JUSTICE ROBERTS   
28   15-1039         JUSTICE BREYER   
29   15-1039       JUSTICE GINSBURG   
..       ...                    ...   
416    16-54  CHIEF JUSTICE ROBERTS   
417    16-54          JUSTICE ALITO   
418    16-54         JUSTICE BREYER   
419    16-54       JUSTICE GINSBURG   
420    16-54          JUSTICE KAGAN   
421    16-54        JUSTICE KENNEDY   
422    16-54      JUSTICE SOTOMAYOR   
423   16-605  CHIEF JUSTICE ROBERTS   
424   16-605          JUSTICE ALITO   
425   16-605         JUSTICE BREYER   
426   16-605       JUSTICE GINSBURG   
427   16-605        JUSTICE GORSUCH   
428   16-605          JUSTICE KAGAN   
429   16-605        JUSTICE KENNEDY   
430   16-605      JUSTICE SOTOMAYOR   
431  16-6219  CHIEF JUSTICE ROBERTS   
432  16-6219          JUSTICE ALITO   
433  16-6219         JUSTICE BREYER   
434  16-6219       JUSTICE GINSBURG   
435  16-6219        JUSTICE GORSUCH   
436  16-6219          JUSTICE KAGAN   
437  16-6219        JUSTICE KENNEDY   
438  16-6219      JUSTICE SOTOMAYOR   
439    16-74  CHIEF JUSTICE ROBERTS   
440    16-74          JUSTICE ALITO   
441    16-74         JUSTICE BREYER   
442    16-74       JUSTICE GINSBURG   
443    16-74          JUSTICE KAGAN   
444    16-74        JUSTICE KENNEDY   
445    16-74      JUSTICE SOTOMAYOR   

                                                 lines  \
0    <p>Thank you, counsel.\n\n The case is submitt...   
1    <p>It comes out the other way.  And the Red Cr...   
2    <p>But Red Cross didn't have  the competence l...   
3    <p>Well, suppose it means  personal jurisdicti...   
4    <p>Don't tell us we're not\n\n working hard en...   
5    <p>Isn't that USG's\n\n position in Portsmouth...   
6    <p>Thank you, counsel.  The case is submitted....   
7    <p>Well, in order to do that,  it would be nec...   
8    <p>So they are worried about\n\n it, in other ...   
9    <p>And so you agree that  "substantial portion...   
10   <p>But then you take Justice  Alito's hypothet...   
11   <p>Under the U.K. laws, are  patents generally...   
12   <p>Let's assume the -</p><p> Sure.</p><p> It s...   
13   <p>Thank you, counsel.  The case is submitted....   
14   <p>On page 30 of your brief.\n\n 30 to 31 of y...   
15   <p>That's right. I mean, I\n\n may be missing ...   
16   <p>But it's a different  claim. One claim is f...   
17   <p>Mr. Eisenhammer, why should\n\n we even get...   
18   <p>Okay. It's terminated six\n\n months -- or ...   
19   <p>So you -- you don't care  that we don't rea...   
20   <p>Thank you, counsel.\n\n Mr. Wirken.\n\n ORA...   
21   <p>I think the question is can\n\n a State dep...   
22   <p>All right. That's the only\n\n difference. ...   
23   <p>What would be\n\n noncomplying with Federal...   
24   <p>But sorry to come back to  this, Mr. Unikow...   
25   <p>Is -- is the answer that\n\n there's a diff...   
26   <p>So could she 

In [420]:
result_list = []
for item in lines_out['docket'].unique():
    result = {}
    for x in lines_out[lines_out['docket'] == item]['line_dict']:
        result.update(x)
    result_list.append(result)
print(result_list)

[{'CHIEF JUSTICE ROBERTS': "<p>Thank you, counsel.\n\n The case is submitted.\n\n (Whereupon, at :02 p.m., the case in the\n\n above-entitled matter was submitted.)</p><p> Thank you, counsel.\n\n Mr. Rosenkranz, three minutes.\n\n REBUTTAL ARGUMENT OF E. JOSHUA ROSENKRANZ\n\n ON BEHALF OF THE PETITIONERS</p><p> Do you know why they  have that different structure on the board of directors?</p><p> Well, there is a\n\n reason -- your -- your friend on the other side suggests\n\n there was a reason for treating them differently, which\n\n is that the board -- the Freddie board is -- consists of\n\n Federal officers, while the idea, of course, is that\n\n Fannie's board, two-thirds of it does not.</p><p> He was fired.</p><p> The -- the Federal\n \n39  courts -- may be stepping back a bit. There are courts  of limited jurisdiction. And to get into them you have  to carry, I think, a significant burden to establish  their right to be there.  Now, I think you have to do more than win  51/49, g

In [421]:
len(result_list)

64

In [ ]:
lines_out.apply(lambda x: "%s" % '</p><p> '.join(x)).reset_index(name='lines')

In [129]:
df4 = df3.groupby('docket')['speaker'].value_counts().reset_index(name = 'per_docket')

docket                speaker  per_docket
0    14-1055         JUSTICE BREYER          25
1    14-1055      JUSTICE SOTOMAYOR          17
2    14-1055       JUSTICE GINSBURG          14
3    14-1055  CHIEF JUSTICE ROBERTS          13
4    14-1055        JUSTICE KENNEDY           2
5    14-1055          JUSTICE KAGAN           1
6    14-1538         JUSTICE BREYER          20
7    14-1538  CHIEF JUSTICE ROBERTS          16
8    14-1538      JUSTICE SOTOMAYOR          15
9    14-1538       JUSTICE GINSBURG          13
10   14-1538        JUSTICE KENNEDY          11
11   14-1538          JUSTICE ALITO           9
12   14-1538          JUSTICE KAGAN           8
13   14-9496        JUSTICE KENNEDY          21
14   14-9496  CHIEF JUSTICE ROBERTS          18
15   14-9496      JUSTICE SOTOMAYOR          18
16   14-9496          JUSTICE ALITO          14
17   14-9496         JUSTICE BREYER          14
18   14-9496          JUSTICE KAGAN          12
19   14-9496       JUSTICE GINSBURG           7
20   15-1031  CHIEF JUSTICE ROBERTS          30
21   15-1031         JUSTICE BREYER          27
22   15-1031      JUSTICE SOTOMAYOR          18
23   15-1031       JUSTICE GINSBURG          11
24   15-1031          JUSTICE KAGAN           5
25   15-1031        JUSTICE KENNEDY           4
26   15-1031          JUSTICE ALITO           2
27   15-1039      JUSTICE SOTOMAYOR          49
28   15-1039         JUSTICE BREYER          24
29   15-1039  CHIEF JUSTICE ROBERTS          22
..       ...                    ...         ...
416    16-54          JUSTICE KAGAN          40
417    16-54         JUSTICE BREYER          20
418    16-54      JUSTICE SOTOMAYOR          14
419    16-54          JUSTICE ALITO          12
420    16-54  CHIEF JUSTICE ROBERTS          10
421    16-54       JUSTICE GINSBURG          10
422    16-54        JUSTICE KENNEDY           3
423   16-605  CHIEF JUSTICE ROBERTS          30
424   16-605        JUSTICE GORSUCH          20
425   16-605         JUSTICE BREYER          19
426   16-605        JUSTICE KENNEDY          13
427   16-605      JUSTICE SOTOMAYOR          13
428   16-605       JUSTICE GINSBURG          11
429   16-605          JUSTICE ALITO          10
430   16-605          JUSTICE KAGAN           6
431  16-6219      JUSTICE SOTOMAYOR          24
432  16-6219         JUSTICE BREYER          23
433  16-6219          JUSTICE KAGAN          23
434  16-6219  CHIEF JUSTICE ROBERTS          18
435  16-6219        JUSTICE GORSUCH           7
436  16-6219        JUSTICE KENNEDY           7
437  16-6219          JUSTICE ALITO           6
438  16-6219       JUSTICE GINSBURG           5
439    16-74  CHIEF JUSTICE ROBERTS          38
440    16-74          JUSTICE ALITO          26
441    16-74      JUSTICE SOTOMAYOR          20
442    16-74         JUSTICE BREYER          18
443    16-74          JUSTICE KAGAN           9
444    16-74        JUSTICE KENNEDY           9
445    16-74       JUSTICE GINSBURG           8

[446 rows x 3 columns]

In [392]:
df4['string'] = df4['speaker'].str.title() + ": " + df4['per_docket'].map(str)
df4

docket                speaker  per_docket                     string
0    14-1055         JUSTICE BREYER          25         Justice Breyer: 25
1    14-1055      JUSTICE SOTOMAYOR          17      Justice Sotomayor: 17
2    14-1055       JUSTICE GINSBURG          14       Justice Ginsburg: 14
3    14-1055  CHIEF JUSTICE ROBERTS          13  Chief Justice Roberts: 13
4    14-1055        JUSTICE KENNEDY           2         Justice Kennedy: 2
5    14-1055          JUSTICE KAGAN           1           Justice Kagan: 1
6    14-1538         JUSTICE BREYER          20         Justice Breyer: 20
7    14-1538  CHIEF JUSTICE ROBERTS          16  Chief Justice Roberts: 16
8    14-1538      JUSTICE SOTOMAYOR          15      Justice Sotomayor: 15
9    14-1538       JUSTICE GINSBURG          13       Justice Ginsburg: 13
10   14-1538        JUSTICE KENNEDY          11        Justice Kennedy: 11
11   14-1538          JUSTICE ALITO           9           Justice Alito: 9
12   14-1538          JUSTICE KAGAN           8           Justice Kagan: 8
13   14-9496        JUSTICE KENNEDY          21        Justice Kennedy: 21
14   14-9496  CHIEF JUSTICE ROBERTS          18  Chief Justice Roberts: 18
15   14-9496      JUSTICE SOTOMAYOR          18      Justice Sotomayor: 18
16   14-9496          JUSTICE ALITO          14          Justice Alito: 14
17   14-9496         JUSTICE BREYER          14         Justice Breyer: 14
18   14-9496          JUSTICE KAGAN          12          Justice Kagan: 12
19   14-9496       JUSTICE GINSBURG           7        Justice Ginsburg: 7
20   15-1031  CHIEF JUSTICE ROBERTS          30  Chief Justice Roberts: 30
21   15-1031         JUSTICE BREYER          27         Justice Breyer: 27
22   15-1031      JUSTICE SOTOMAYOR          18      Justice Sotomayor: 18
23   15-1031       JUSTICE GINSBURG          11       Justice Ginsburg: 11
24   15-1031          JUSTICE KAGAN           5           Justice Kagan: 5
25   15-1031        JUSTICE KENNEDY           4         Justice Kennedy: 4
26   15-1031          JUSTICE ALITO           2           Justice Alito: 2
27   15-1039      JUSTICE SOTOMAYOR          49      Justice Sotomayor: 49
28   15-1039         JUSTICE BREYER          24         Justice Breyer: 24
29   15-1039  CHIEF JUSTICE ROBERTS          22  Chief Justice Roberts: 22
..       ...                    ...         ...                        ...
416    16-54          JUSTICE KAGAN          40          Justice Kagan: 40
417    16-54         JUSTICE BREYER          20         Justice Breyer: 20
418    16-54      JUSTICE SOTOMAYOR          14      Justice Sotomayor: 14
419    16-54          JUSTICE ALITO          12          Justice Alito: 12
420    16-54  CHIEF JUSTICE ROBERTS          10  Chief Justice Roberts: 10
421    16-54       JUSTICE GINSBURG          10       Justice Ginsburg: 10
422    16-54        JUSTICE KENNEDY           3         Justice Kennedy: 3
423   16-605  CHIEF JUSTICE ROBERTS          30  Chief Justice Roberts: 30
424   16-605        JUSTICE GORSUCH          20        Justice Gorsuch: 20
425   16-605         JUSTICE BREYER          19         Justice Breyer: 19
426   16-605        JUSTICE KENNEDY          13        Justice Kennedy: 13
427   16-605      JUSTICE SOTOMAYOR          13      Justice Sotomayor: 13
428   16-605       JUSTICE GINSBURG          11       Justice Ginsburg: 11
429   16-605          JUSTICE ALITO          10          Justice Alito: 10
430   16-605          JUSTICE KAGAN           6           Justice Kagan: 6
431  16-6219      JUSTICE SOTOMAYOR          24      Justice Sotomayor: 24
432  16-6219         JUSTICE BREYER          23         Justice Breyer: 23
433  16-6219          JUSTICE KAGAN          23          Justice Kagan: 23
434  16-6219  CHIEF JUSTICE ROBERTS          18  Chief Justice Roberts: 18
435  16-6219        JUSTICE GORSUCH           7         Justice Gorsuch: 7
436  16-6219        JUSTICE KENNEDY           7         Justice Kennedy: 7
437  16-6219          JUSTICE A

In [393]:
output = df4.groupby('docket')['string'].apply(lambda x: ",  ".join(x)).reset_index(name='article')
output2 = df4.groupby('docket')['string'].apply(lambda x: "<p>%s</p>" % '</p><p> '.join(x)).reset_index(name='article')
output

docket                                            article
0   14-1055  Justice Breyer: 25,  Justice Sotomayor: 17,  J...
1   14-1538  Justice Breyer: 20,  Chief Justice Roberts: 16...
2   14-9496  Justice Kennedy: 21,  Chief Justice Roberts: 1...
3   15-1031  Chief Justice Roberts: 30,  Justice Breyer: 27...
4   15-1039  Justice Sotomayor: 49,  Justice Breyer: 24,  C...
5   15-1111  Chief Justice Roberts: 26,  Justice Kennedy: 1...
6    15-118  Justice Breyer: 28,  Justice Kagan: 21,  Justi...
7   15-1189  Justice Breyer: 20,  Chief Justice Roberts: 10...
8   15-1191  Justice Breyer: 36,  Justice Ginsburg: 27,  Ju...
9   15-1194  Justice Breyer: 26,  Justice Kagan: 25,  Justi...
10  15-1204  Justice Breyer: 46,  Justice Sotomayor: 42,  J...
11  15-1248  Justice Breyer: 26,  Justice Sotomayor: 21,  C...
12  15-1251  Justice Kagan: 21,  Justice Ginsburg: 10,  Chi...
13  15-1256  Justice Sotomayor: 26,  Justice Breyer: 25,  C...
14  15-1262  Justice Breyer: 20,  Justice Kagan: 15,  Chief...
15  15-1293  Justice Kagan: 27,  Chief Justice Roberts: 19,...
16  15-1358  Justice Breyer: 26,  Chief Justice Roberts: 16...
17  15-1391  Justice Breyer: 28,  Justice Kagan: 26,  Justi...
18  15-1406  Chief Justice Roberts: 19,  Justice Kagan: 19,...
19  15-1498  Justice Kagan: 28,  Justice Alito: 13,  Justic...
20  15-1500  Justice Breyer: 22,  Justice Ginsburg: 19,  Ju...
21  15-1503  Justice Ginsburg: 11,  Justice Kagan: 8,  Just...
22   15-214  Chief Justice Roberts: 36,  Justice Kennedy: 3...
23   15-423  Chief Justice Roberts: 23,  Justice Kagan: 20,...
24   15-457  Justice Ginsburg: 28,  Chief Justice Roberts: ...
25   15-497  Justice Sotomayor: 40,  Chief Justice Roberts:...
26   15-513  Justice Breyer: 17,  Chief Justice Roberts: 13...
27   15-537  Justice Kagan: 20,  Justice Kennedy: 17,  Just...
28   15-577  Justice Breyer: 31,  Justice Kagan: 26,  Justi...
29  15-5991  Justice Kagan: 36,  Justice Breyer: 31,  Justi...
..      ...                                                ...
34  15-7250  Chief Justice Roberts: 26,  Justice Kagan: 20,...
35   15-777  Justice Sotomayor: 19,  Justice Kennedy: 18,  ...
36   15-797  Justice Sotomayor: 26,  Justice Kagan: 18,  Ju...
37  15-8049  Chief Justice Roberts: 27,  Justice Sotomayor:...
38   15-827  Justice Breyer: 38,  Chief Justice Roberts: 27...
39  15-8544  Justice Sotomayor: 27,  Justice Kennedy: 15,  ...
40   15-866  Justice Breyer: 32,  Justice Kagan: 31,  Justi...
41  15-9260  Justice Breyer: 40,  Chief Justice Roberts: 18...
42   15-927  Justice Breyer: 20,  Justice Ginsburg: 19,  Ch...
43   16-142  Chief Justice Roberts: 24,  Justice Breyer: 24...
44   16-149  Chief Justice Roberts: 22,  Justice Alito: 15,...
45   16-240  Chief Justice Roberts: 21,  Justice Alito: 19,...
46   16-254  Justice Sotomayor: 19,  Justice Ginsburg: 14, ...
47   16-309  Justice Kagan: 35,  Chief Justice Roberts: 25,...
48    16-32  Chief Justice Roberts: 27,  Justice Breyer: 23...
49   16-327  Justice Kagan: 33,  Justice Sotomayor: 33,  Ju...
50   16-341  Justice Breyer: 38,  Chief Justice Roberts: 20...
51   16-348  Justice Breyer: 28,  Justice Sotomayor: 25,  J...
52   16-349  Chief Justice Roberts: 13,  Justice Alito: 9, ...
53   16-369  Chief Justice Roberts: 35,  Justice Kagan: 29,...
54   16-373  Justice Sotomayor: 28,  Justice Alito: 13,  Ju...
55   16-399  Justice Gorsuch: 47,  Justice Sotomayor: 28,  ...
56   16-405  Chief Justice Roberts: 30,  Justice Sotomayor:...
57   16-466  Justice Sotomayor: 32,  Justice Breyer: 24,  J...
58   16-529  Chief Justice Roberts: 32,  Justice Sotomayor:...
59  16-5294  Justice Breyer: 35,  Justice Sotomayor: 15,  C...
60    16-54  Justice Kagan: 40,  Justice Breyer: 20,  Justi...
61   16-605  Chief Justice Roberts: 30,  Justice Gorsuch: 2...
62  16-6219  Justice Sotomayor: 24,  Justice Breyer: 23,  J...
63    16-74  Chief Justice Roberts: 38,  Justice Alito: 26,...

[64 rows x 2 columns]

In [423]:
output['f_speaker_line'] = result_list
output

docket                                            article  \
0   14-1055  Justice Breyer: 25,  Justice Sotomayor: 17,  J...   
1   14-1538  Justice Breyer: 20,  Chief Justice Roberts: 16...   
2   14-9496  Justice Kennedy: 21,  Chief Justice Roberts: 1...   
3   15-1031  Chief Justice Roberts: 30,  Justice Breyer: 27...   
4   15-1039  Justice Sotomayor: 49,  Justice Breyer: 24,  C...   
5   15-1111  Chief Justice Roberts: 26,  Justice Kennedy: 1...   
6    15-118  Justice Breyer: 28,  Justice Kagan: 21,  Justi...   
7   15-1189  Justice Breyer: 20,  Chief Justice Roberts: 10...   
8   15-1191  Justice Breyer: 36,  Justice Ginsburg: 27,  Ju...   
9   15-1194  Justice Breyer: 26,  Justice Kagan: 25,  Justi...   
10  15-1204  Justice Breyer: 46,  Justice Sotomayor: 42,  J...   
11  15-1248  Justice Breyer: 26,  Justice Sotomayor: 21,  C...   
12  15-1251  Justice Kagan: 21,  Justice Ginsburg: 10,  Chi...   
13  15-1256  Justice Sotomayor: 26,  Justice Breyer: 25,  C...   
14  15-1262  Justice Breyer: 20,  Justice Kagan: 15,  Chief...   
15  15-1293  Justice Kagan: 27,  Chief Justice Roberts: 19,...   
16  15-1358  Justice Breyer: 26,  Chief Justice Roberts: 16...   
17  15-1391  Justice Breyer: 28,  Justice Kagan: 26,  Justi...   
18  15-1406  Chief Justice Roberts: 19,  Justice Kagan: 19,...   
19  15-1498  Justice Kagan: 28,  Justice Alito: 13,  Justic...   
20  15-1500  Justice Breyer: 22,  Justice Ginsburg: 19,  Ju...   
21  15-1503  Justice Ginsburg: 11,  Justice Kagan: 8,  Just...   
22   15-214  Chief Justice Roberts: 36,  Justice Kennedy: 3...   
23   15-423  Chief Justice Roberts: 23,  Justice Kagan: 20,...   
24   15-457  Justice Ginsburg: 28,  Chief Justice Roberts: ...   
25   15-497  Justice Sotomayor: 40,  Chief Justice Roberts:...   
26   15-513  Justice Breyer: 17,  Chief Justice Roberts: 13...   
27   15-537  Justice Kagan: 20,  Justice Kennedy: 17,  Just...   
28   15-577  Justice Breyer: 31,  Justice Kagan: 26,  Justi...   
29  15-5991  Justice Kagan: 36,  Justice Breyer: 31,  Justi...   
..      ...                                                ...   
34  15-7250  Chief Justice Roberts: 26,  Justice Kagan: 20,...   
35   15-777  Justice Sotomayor: 19,  Justice Kennedy: 18,  ...   
36   15-797  Justice Sotomayor: 26,  Justice Kagan: 18,  Ju...   
37  15-8049  Chief Justice Roberts: 27,  Justice Sotomayor:...   
38   15-827  Justice Breyer: 38,  Chief Justice Roberts: 27...   
39  15-8544  Justice Sotomayor: 27,  Justice Kennedy: 15,  ...   
40   15-866  Justice Breyer: 32,  Justice Kagan: 31,  Justi...   
41  15-9260  Justice Breyer: 40,  Chief Justice Roberts: 18...   
42   15-927  Justice Breyer: 20,  Justice Ginsburg: 19,  Ch...   
43   16-142  Chief Justice Roberts: 24,  Justice Breyer: 24...   
44   16-149  Chief Justice Roberts: 22,  Justice Alito: 15,...   
45   16-240  Chief Justice Roberts: 21,  Justice Alito: 19,...   
46   16-254  Justice Sotomayor: 19,  Justice Ginsburg: 14, ...   
47   16-309  Justice Kagan: 35,  Chief Justice Roberts: 25,...   
48    16-32  Chief Justice Roberts: 27,  Justice Breyer: 23...   
49   16-327  Justice Kagan: 33,  Justice Sotomayor: 33,  Ju...   
50   16-341  Justice Breyer: 38,  Chief Justice Roberts: 20...   
51   16-348  Justice Breyer: 28,  Justice Sotomayor: 25,  J...   
52   16-349  Chief Justice Roberts: 13,  Justice Alito: 9, ...   
53   16-369  Chief Justice Roberts: 35,  Justice Kagan: 29,...   
54   16-373  Justice Sotomayor: 28,  Justice Alito: 13,  Ju...   
55   16-399  Justice Gorsuch: 47,  Justice Sotomayor: 28,  ...   
56   16-405  Chief Justice Roberts: 30,  Justice Sotomayor:...   
57   16-466  Justice Sotomayor: 32,  Justice Breyer: 24,  J...   
58   16-529  Chief Justice Roberts: 32,  Justice Sotomayor:...   
59  16-5294  Justice Breyer: 35,  Justice Sotomayor: 15,  C...   
60    16-54  Justice Kagan: 40,  Justice Breyer: 20,  Justi...   
61   16-605  Chief Justice Roberts: 30,  Justice Gorsuch: 2...   
62  16-6219  Justice Sotomayor: 24,  Justice 

In [62]:
df3['speaker'].value_counts()

JUSTICE BREYER           1301
CHIEF JUSTICE ROBERTS    1277
JUSTICE SOTOMAYOR        1225
JUSTICE KAGAN            1043
JUSTICE GINSBURG          661
JUSTICE KENNEDY           648
JUSTICE ALITO             623
JUSTICE GORSUCH           171
Name: speaker, dtype: int64

In [188]:
justices = df3.groupby('speaker')['docket'].value_counts().reset_index(name='times')
justices

speaker   docket  times
0    CHIEF JUSTICE ROBERTS    16-74     38
1    CHIEF JUSTICE ROBERTS   15-214     36
2    CHIEF JUSTICE ROBERTS   16-369     35
3    CHIEF JUSTICE ROBERTS   15-497     33
4    CHIEF JUSTICE ROBERTS   16-529     32
5    CHIEF JUSTICE ROBERTS  15-1031     30
6    CHIEF JUSTICE ROBERTS   15-680     30
7    CHIEF JUSTICE ROBERTS   16-405     30
8    CHIEF JUSTICE ROBERTS   16-605     30
9    CHIEF JUSTICE ROBERTS   15-866     29
10   CHIEF JUSTICE ROBERTS  15-8049     27
11   CHIEF JUSTICE ROBERTS   15-827     27
12   CHIEF JUSTICE ROBERTS    16-32     27
13   CHIEF JUSTICE ROBERTS  15-1111     26
14   CHIEF JUSTICE ROBERTS  15-7250     26
15   CHIEF JUSTICE ROBERTS   16-309     25
16   CHIEF JUSTICE ROBERTS  15-1391     24
17   CHIEF JUSTICE ROBERTS   16-142     24
18   CHIEF JUSTICE ROBERTS   15-423     23
19   CHIEF JUSTICE ROBERTS   15-577     23
20   CHIEF JUSTICE ROBERTS   16-399     23
21   CHIEF JUSTICE ROBERTS  15-1039     22
22   CHIEF JUSTICE ROBERTS   15-457     22
23   CHIEF JUSTICE ROBERTS   15-628     22
24   CHIEF JUSTICE ROBERTS   16-149     22
25   CHIEF JUSTICE ROBERTS  15-1256     21
26   CHIEF JUSTICE ROBERTS  15-5991     21
27   CHIEF JUSTICE ROBERTS   16-240     21
28   CHIEF JUSTICE ROBERTS   16-348     21
29   CHIEF JUSTICE ROBERTS   15-649     20
..                     ...      ...    ...
416      JUSTICE SOTOMAYOR  14-9496     18
417      JUSTICE SOTOMAYOR  15-1031     18
418      JUSTICE SOTOMAYOR  15-1500     18
419      JUSTICE SOTOMAYOR   16-369     18
420      JUSTICE SOTOMAYOR  14-1055     17
421      JUSTICE SOTOMAYOR   15-118     16
422      JUSTICE SOTOMAYOR   15-827     16
423      JUSTICE SOTOMAYOR  15-9260     16
424      JUSTICE SOTOMAYOR   16-405     16
425      JUSTICE SOTOMAYOR  14-1538     15
426      JUSTICE SOTOMAYOR  16-5294     15
427      JUSTICE SOTOMAYOR  15-1406     14
428      JUSTICE SOTOMAYOR   16-240     14
429      JUSTICE SOTOMAYOR    16-54     14
430      JUSTICE SOTOMAYOR   16-605     13
431      JUSTICE SOTOMAYOR  15-1293     12
432      JUSTICE SOTOMAYOR   15-423     12
433      JUSTICE SOTOMAYOR   15-927     12
434      JUSTICE SOTOMAYOR  15-1498     11
435      JUSTICE SOTOMAYOR   16-341     11
436      JUSTICE SOTOMAYOR  15-1191     10
437      JUSTICE SOTOMAYOR   16-149     10
438      JUSTICE SOTOMAYOR  15-1262      8
439      JUSTICE SOTOMAYOR  15-1503      8
440      JUSTICE SOTOMAYOR    16-32      8
441      JUSTICE SOTOMAYOR  15-1251      7
442      JUSTICE SOTOMAYOR  15-1189      6
443      JUSTICE SOTOMAYOR   15-513      6
444      JUSTICE SOTOMAYOR   15-537      4
445      JUSTICE SOTOMAYOR   16-349      4

[446 rows x 3 columns]

In [220]:
justices2 = output2
justices2

docket                                            article
0   14-1055  <p>JUSTICE BREYER: 25</p><p> JUSTICE SOTOMAYOR...
1   14-1538  <p>JUSTICE BREYER: 20</p><p> CHIEF JUSTICE ROB...
2   14-9496  <p>JUSTICE KENNEDY: 21</p><p> CHIEF JUSTICE RO...
3   15-1031  <p>CHIEF JUSTICE ROBERTS: 30</p><p> JUSTICE BR...
4   15-1039  <p>JUSTICE SOTOMAYOR: 49</p><p> JUSTICE BREYER...
5   15-1111  <p>CHIEF JUSTICE ROBERTS: 26</p><p> JUSTICE KE...
6    15-118  <p>JUSTICE BREYER: 28</p><p> JUSTICE KAGAN: 21...
7   15-1189  <p>JUSTICE BREYER: 20</p><p> CHIEF JUSTICE ROB...
8   15-1191  <p>JUSTICE BREYER: 36</p><p> JUSTICE GINSBURG:...
9   15-1194  <p>JUSTICE BREYER: 26</p><p> JUSTICE KAGAN: 25...
10  15-1204  <p>JUSTICE BREYER: 46</p><p> JUSTICE SOTOMAYOR...
11  15-1248  <p>JUSTICE BREYER: 26</p><p> JUSTICE SOTOMAYOR...
12  15-1251  <p>JUSTICE KAGAN: 21</p><p> JUSTICE GINSBURG: ...
13  15-1256  <p>JUSTICE SOTOMAYOR: 26</p><p> JUSTICE BREYER...
14  15-1262  <p>JUSTICE BREYER: 20</p><p> JUSTICE KAGAN: 15...
15  15-1293  <p>JUSTICE KAGAN: 27</p><p> CHIEF JUSTICE ROBE...
16  15-1358  <p>JUSTICE BREYER: 26</p><p> CHIEF JUSTICE ROB...
17  15-1391  <p>JUSTICE BREYER: 28</p><p> JUSTICE KAGAN: 26...
18  15-1406  <p>CHIEF JUSTICE ROBERTS: 19</p><p> JUSTICE KA...
19  15-1498  <p>JUSTICE KAGAN: 28</p><p> JUSTICE ALITO: 13<...
20  15-1500  <p>JUSTICE BREYER: 22</p><p> JUSTICE GINSBURG:...
21  15-1503  <p>JUSTICE GINSBURG: 11</p><p> JUSTICE KAGAN: ...
22   15-214  <p>CHIEF JUSTICE ROBERTS: 36</p><p> JUSTICE KE...
23   15-423  <p>CHIEF JUSTICE ROBERTS: 23</p><p> JUSTICE KA...
24   15-457  <p>JUSTICE GINSBURG: 28</p><p> CHIEF JUSTICE R...
25   15-497  <p>JUSTICE SOTOMAYOR: 40</p><p> CHIEF JUSTICE ...
26   15-513  <p>JUSTICE BREYER: 17</p><p> CHIEF JUSTICE ROB...
27   15-537  <p>JUSTICE KAGAN: 20</p><p> JUSTICE KENNEDY: 1...
28   15-577  <p>JUSTICE BREYER: 31</p><p> JUSTICE KAGAN: 26...
29  15-5991  <p>JUSTICE KAGAN: 36</p><p> JUSTICE BREYER: 31...
..      ...                                                ...
34  15-7250  <p>CHIEF JUSTICE ROBERTS: 26</p><p> JUSTICE KA...
35   15-777  <p>JUSTICE SOTOMAYOR: 19</p><p> JUSTICE KENNED...
36   15-797  <p>JUSTICE SOTOMAYOR: 26</p><p> JUSTICE KAGAN:...
37  15-8049  <p>CHIEF JUSTICE ROBERTS: 27</p><p> JUSTICE SO...
38   15-827  <p>JUSTICE BREYER: 38</p><p> CHIEF JUSTICE ROB...
39  15-8544  <p>JUSTICE SOTOMAYOR: 27</p><p> JUSTICE KENNED...
40   15-866  <p>JUSTICE BREYER: 32</p><p> JUSTICE KAGAN: 31...
41  15-9260  <p>JUSTICE BREYER: 40</p><p> CHIEF JUSTICE ROB...
42   15-927  <p>JUSTICE BREYER: 20</p><p> JUSTICE GINSBURG:...
43   16-142  <p>CHIEF JUSTICE ROBERTS: 24</p><p> JUSTICE BR...
44   16-149  <p>CHIEF JUSTICE ROBERTS: 22</p><p> JUSTICE AL...
45   16-240  <p>CHIEF JUSTICE ROBERTS: 21</p><p> JUSTICE AL...
46   16-254  <p>JUSTICE SOTOMAYOR: 19</p><p> JUSTICE GINSBU...
47   16-309  <p>JUSTICE KAGAN: 35</p><p> CHIEF JUSTICE ROBE...
48    16-32  <p>CHIEF JUSTICE ROBERTS: 27</p><p> JUSTICE BR...
49   16-327  <p>JUSTICE KAGAN: 33</p><p> JUSTICE SOTOMAYOR:...
50   16-341  <p>JUSTICE BREYER: 38</p><p> CHIEF JUSTICE ROB...
51   16-348  <p>JUSTICE BREYER: 28</p><p> JUSTICE SOTOMAYOR...
52   16-349  <p>CHIEF JUSTICE ROBERTS: 13</p><p> JUSTICE AL...
53   16-369  <p>CHIEF JUSTICE ROBERTS: 35</p><p> JUSTICE KA...
54   16-373  <p>JUSTICE SOTOMAYOR: 28</p><p> JUSTICE ALITO:...
55   16-399  <p>JUSTICE GORSUCH: 47</p><p> JUSTICE SOTOMAYO...
56   16-405  <p>CHIEF JUSTICE ROBERTS: 30</p><p> JUSTICE SO...
57   16-466  <p>JUSTICE SOTOMAYOR: 32</p><p> JUSTICE BREYER...
58   16-529  <p>CHIEF JUSTICE ROBERTS: 32</p><p> JUSTICE SO...
59  16-5294  <p>JUSTICE BREYER: 35</p><p> JUSTICE SOTOMAYOR...
60    16-54  <p>JUSTICE KAGAN: 40</p><p> JUSTICE BREYER: 20...
61   16-605  <p>CHIEF JUSTICE ROBERTS: 30</p><p> JUSTICE GO...
62  16-6219  <p>JUSTICE SOTOMAYOR: 24</p><p> JUSTICE BREYER...
63    16-74  <p>CHIEF JUSTICE ROBERTS: 38</p><p> JUSTICE AL...

[64 rows x 2 columns]

In [288]:
justices2['speak_in'] = '0'
justices2['color'] = 'CE21C3'
justices2 = justices2[['docket', 'speak_in', 'color']]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [289]:
justices2

docket speak_in   color
0   14-1055        0  CE21C3
1   14-1538        0  CE21C3
2   14-9496        0  CE21C3
3   15-1031        0  CE21C3
4   15-1039        0  CE21C3
5   15-1111        0  CE21C3
6    15-118        0  CE21C3
7   15-1189        0  CE21C3
8   15-1191        0  CE21C3
9   15-1194        0  CE21C3
10  15-1204        0  CE21C3
11  15-1248        0  CE21C3
12  15-1251        0  CE21C3
13  15-1256        0  CE21C3
14  15-1262        0  CE21C3
15  15-1293        0  CE21C3
16  15-1358        0  CE21C3
17  15-1391        0  CE21C3
18  15-1406        0  CE21C3
19  15-1498        0  CE21C3
20  15-1500        0  CE21C3
21  15-1503        0  CE21C3
22   15-214        0  CE21C3
23   15-423        0  CE21C3
24   15-457        0  CE21C3
25   15-497        0  CE21C3
26   15-513        0  CE21C3
27   15-537        0  CE21C3
28   15-577        0  CE21C3
29  15-5991        0  CE21C3
..      ...      ...     ...
34  15-7250        0  CE21C3
35   15-777        0  CE21C3
36   15-797        0  CE21C3
37  15-8049        0  CE21C3
38   15-827        0  CE21C3
39  15-8544        0  CE21C3
40   15-866        0  CE21C3
41  15-9260        0  CE21C3
42   15-927        0  CE21C3
43   16-142        0  CE21C3
44   16-149        0  CE21C3
45   16-240        0  CE21C3
46   16-254        0  CE21C3
47   16-309        0  CE21C3
48    16-32        0  CE21C3
49   16-327        0  CE21C3
50   16-341        0  CE21C3
51   16-348        0  CE21C3
52   16-349        0  CE21C3
53   16-369        0  CE21C3
54   16-373        0  CE21C3
55   16-399        0  CE21C3
56   16-405        0  CE21C3
57   16-466        0  CE21C3
58   16-529        0  CE21C3
59  16-5294        0  CE21C3
60    16-54        0  CE21C3
61   16-605        0  CE21C3
62  16-6219        0  CE21C3
63    16-74        0  CE21C3

[64 rows x 3 columns]

In [286]:
justices.loc[justices.speaker == 'CHIEF JUSTICE ROBERTS', 'speak_in'] = '1'
justices.loc[justices.speaker == 'JUSTICE ALITO', 'speak_in'] = '2'
justices.loc[justices.speaker == 'JUSTICE BREYER', 'speak_in'] = '3'
justices.loc[justices.speaker == 'JUSTICE GINSBURG', 'speak_in'] = '4'
justices.loc[justices.speaker == 'JUSTICE GORSUCH', 'speak_in'] = '5'
justices.loc[justices.speaker == 'JUSTICE KAGAN', 'speak_in'] = '6'
justices.loc[justices.speaker == 'JUSTICE KENNEDY', 'speak_in'] = '7'
justices.loc[justices.speaker == 'JUSTICE SOTOMAYOR', 'speak_in'] = '8'

In [331]:
justices

speaker  \
0                                  CHIEF JUSTICE ROBERTS   
1                                  CHIEF JUSTICE ROBERTS   
2                                  CHIEF JUSTICE ROBERTS   
3                                  CHIEF JUSTICE ROBERTS   
4                                  CHIEF JUSTICE ROBERTS   
5                                  CHIEF JUSTICE ROBERTS   
6                                  CHIEF JUSTICE ROBERTS   
7                                  CHIEF JUSTICE ROBERTS   
8                                  CHIEF JUSTICE ROBERTS   
9                                  CHIEF JUSTICE ROBERTS   
10                                 CHIEF JUSTICE ROBERTS   
11                                 CHIEF JUSTICE ROBERTS   
12                                 CHIEF JUSTICE ROBERTS   
13                                 CHIEF JUSTICE ROBERTS   
14                                 CHIEF JUSTICE ROBERTS   
15                                 CHIEF JUSTICE ROBERTS   
16                                 CHIEF JUSTICE ROBERTS   
17                                 CHIEF JUSTICE ROBERTS   
18                                 CHIEF JUSTICE ROBERTS   
19                                 CHIEF JUSTICE ROBERTS   
20                                 CHIEF JUSTICE ROBERTS   
21                                 CHIEF JUSTICE ROBERTS   
22                                 CHIEF JUSTICE ROBERTS   
23                                 CHIEF JUSTICE ROBERTS   
24                                 CHIEF JUSTICE ROBERTS   
25                                 CHIEF JUSTICE ROBERTS   
26                                 CHIEF JUSTICE ROBERTS   
27                                 CHIEF JUSTICE ROBERTS   
28                                 CHIEF JUSTICE ROBERTS   
29                                 CHIEF JUSTICE ROBERTS   
...                                                  ...   
417                                    JUSTICE SOTOMAYOR   
418                                    JUSTICE SOTOMAYOR   
419                                    JUSTICE SOTOMAYOR   
420                                    JUSTICE SOTOMAYOR   
421                                    JUSTICE SOTOMAYOR   
422                                    JUSTICE SOTOMAYOR   
423                                    JUSTICE SOTOMAYOR   
424                                    JUSTICE SOTOMAYOR   
425                                    JUSTICE SOTOMAYOR   
426                                    JUSTICE SOTOMAYOR   
427                                    JUSTICE SOTOMAYOR   
428                                    JUSTICE SOTOMAYOR   
429                                    JUSTICE SOTOMAYOR   
430                                    JUSTICE SOTOMAYOR   
431                                    JUSTICE SOTOMAYOR   
432                                    JUSTICE SOTOMAYOR   
433                                    JUSTICE SOTOMAYOR   
434                                    JUSTICE SOTOMAYOR   
435                                    JUSTICE SOTOMAYOR   
436                                    JUSTICE SOTOMAYOR   
437                                    JUSTICE SOTOMAYOR   
438                                    JUSTICE SOTOMAYOR   
439                                    JUSTICE SOTOMAYOR   
440                                    JUSTICE SOTOMAYOR   
441                                    JUSTICE SOTOMAYOR   
442                                    JUSTICE SOTOMAYOR   
443                                    JUSTICE SOTOMAYOR   
444                                    JUSTICE SOTOMAYOR   
445                                    JUSTICE SOTOMAYOR   
color  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...   

                                                  docket  \
0                                                  16-74   
1                                                 15-214   
2                                                 16-369   
3                                                 15-497   
4                           

In [225]:
speak_perdocket = justices.groupby('docket')['speak_in'].value_counts().reset_index(name='times')
speak_perdocket

docket speak_in  times
0    14-1055        1      1
1    14-1055        3      1
2    14-1055        4      1
3    14-1055        6      1
4    14-1055        7      1
5    14-1055        8      1
6    14-1538        1      1
7    14-1538        2      1
8    14-1538        3      1
9    14-1538        4      1
10   14-1538        6      1
11   14-1538        7      1
12   14-1538        8      1
13   14-9496        1      1
14   14-9496        2      1
15   14-9496        3      1
16   14-9496        4      1
17   14-9496        6      1
18   14-9496        7      1
19   14-9496        8      1
20   15-1031        1      1
21   15-1031        2      1
22   15-1031        3      1
23   15-1031        4      1
24   15-1031        6      1
25   15-1031        7      1
26   15-1031        8      1
27   15-1039        1      1
28   15-1039        3      1
29   15-1039        4      1
..       ...      ...    ...
416    16-54        1      1
417    16-54        2      1
418    16-54        3      1
419    16-54        4      1
420    16-54        6      1
421    16-54        7      1
422    16-54        8      1
423   16-605        1      1
424   16-605        2      1
425   16-605        3      1
426   16-605        4      1
427   16-605        5      1
428   16-605        6      1
429   16-605        7      1
430   16-605        8      1
431  16-6219        1      1
432  16-6219        2      1
433  16-6219        3      1
434  16-6219        4      1
435  16-6219        5      1
436  16-6219        6      1
437  16-6219        7      1
438  16-6219        8      1
439    16-74        1      1
440    16-74        2      1
441    16-74        3      1
442    16-74        4      1
443    16-74        6      1
444    16-74        7      1
445    16-74        8      1

[446 rows x 3 columns]

In [292]:
speak_color = justices.groupby('docket')['color'].value_counts().reset_index(name='times')
speak_color

docket   color  times
0    14-1055  000000      1
1    14-1055  0BD488      1
2    14-1055  37CB00      1
3    14-1055  7E3B0D      1
4    14-1055  8A04D8      1
5    14-1055  FF2A0C      1
6    14-1538  000000      1
7    14-1538  0BB2E8      1
8    14-1538  0BD488      1
9    14-1538  37CB00      1
10   14-1538  7E3B0D      1
11   14-1538  8A04D8      1
12   14-1538  FF2A0C      1
13   14-9496  000000      1
14   14-9496  0BB2E8      1
15   14-9496  0BD488      1
16   14-9496  37CB00      1
17   14-9496  7E3B0D      1
18   14-9496  8A04D8      1
19   14-9496  FF2A0C      1
20   15-1031  000000      1
21   15-1031  0BB2E8      1
22   15-1031  0BD488      1
23   15-1031  37CB00      1
24   15-1031  7E3B0D      1
25   15-1031  8A04D8      1
26   15-1031  FF2A0C      1
27   15-1039  000000      1
28   15-1039  0BD488      1
29   15-1039  37CB00      1
..       ...     ...    ...
416    16-54  000000      1
417    16-54  0BB2E8      1
418    16-54  0BD488      1
419    16-54  37CB00      1
420    16-54  7E3B0D      1
421    16-54  8A04D8      1
422    16-54  FF2A0C      1
423   16-605  000000      1
424   16-605  0BB2E8      1
425   16-605  0BD488      1
426   16-605  37CB00      1
427   16-605  7E3B0D      1
428   16-605  8A04D8      1
429   16-605  D82C86      1
430   16-605  FF2A0C      1
431  16-6219  000000      1
432  16-6219  0BB2E8      1
433  16-6219  0BD488      1
434  16-6219  37CB00      1
435  16-6219  7E3B0D      1
436  16-6219  8A04D8      1
437  16-6219  D82C86      1
438  16-6219  FF2A0C      1
439    16-74  000000      1
440    16-74  0BB2E8      1
441    16-74  0BD488      1
442    16-74  37CB00      1
443    16-74  7E3B0D      1
444    16-74  8A04D8      1
445    16-74  FF2A0C      1

[446 rows x 3 columns]

In [207]:
f_speak_perdocket = pd.concat([speak_perdocket, justices2])
f_speak_perdocket

docket speak_in  times
0   14-1055        1    1.0
1   14-1055        3    1.0
2   14-1055        4    1.0
3   14-1055        6    1.0
4   14-1055        7    1.0
5   14-1055        8    1.0
6   14-1538        1    1.0
7   14-1538        2    1.0
8   14-1538        3    1.0
9   14-1538        4    1.0
10  14-1538        6    1.0
11  14-1538        7    1.0
12  14-1538        8    1.0
13  14-9496        1    1.0
14  14-9496        2    1.0
15  14-9496        3    1.0
16  14-9496        4    1.0
17  14-9496        6    1.0
18  14-9496        7    1.0
19  14-9496        8    1.0
20  15-1031        1    1.0
21  15-1031        2    1.0
22  15-1031        3    1.0
23  15-1031        4    1.0
24  15-1031        6    1.0
25  15-1031        7    1.0
26  15-1031        8    1.0
27  15-1039        1    1.0
28  15-1039        3    1.0
29  15-1039        4    1.0
..      ...      ...    ...
34  15-7250        0    NaN
35   15-777        0    NaN
36   15-797        0    NaN
37  15-8049        0    NaN
38   15-827        0    NaN
39  15-8544        0    NaN
40   15-866        0    NaN
41  15-9260        0    NaN
42   15-927        0    NaN
43   16-142        0    NaN
44   16-149        0    NaN
45   16-240        0    NaN
46   16-254        0    NaN
47   16-309        0    NaN
48    16-32        0    NaN
49   16-327        0    NaN
50   16-341        0    NaN
51   16-348        0    NaN
52   16-349        0    NaN
53   16-369        0    NaN
54   16-373        0    NaN
55   16-399        0    NaN
56   16-405        0    NaN
57   16-466        0    NaN
58   16-529        0    NaN
59  16-5294        0    NaN
60    16-54        0    NaN
61   16-605        0    NaN
62  16-6219        0    NaN
63    16-74        0    NaN

[510 rows x 3 columns]

In [298]:
f_speak_color = pd.concat([speak_color, justices2])
f_speak_color.groupby('docket')['color'].value_counts()

docket   color 
14-1055  000000    1
         0BD488    1
         37CB00    1
         7E3B0D    1
         8A04D8    1
         CE21C3    1
         FF2A0C    1
14-1538  000000    1
         0BB2E8    1
         0BD488    1
         37CB00    1
         7E3B0D    1
         8A04D8    1
         CE21C3    1
         FF2A0C    1
14-9496  000000    1
         0BB2E8    1
         0BD488    1
         37CB00    1
         7E3B0D    1
         8A04D8    1
         CE21C3    1
         FF2A0C    1
15-1031  000000    1
         0BB2E8    1
         0BD488    1
         37CB00    1
         7E3B0D    1
         8A04D8    1
         CE21C3    1
                  ..
16-54    7E3B0D    1
         8A04D8    1
         CE21C3    1
         FF2A0C    1
16-605   000000    1
         0BB2E8    1
         0BD488    1
         37CB00    1
         7E3B0D    1
         8A04D8    1
         CE21C3    1
         D82C86    1
         FF2A0C    1
16-6219  000000    1
         0BB2E8    1
         0BD488   

In [226]:
a = f_speak_perdocket.groupby('docket')['speak_in'].apply(lambda x: ','.join(x).split(',')).reset_index(name='speak_list')
a

docket                   speak_list
0   14-1055        [1, 3, 4, 6, 7, 8, 0]
1   14-1538     [1, 2, 3, 4, 6, 7, 8, 0]
2   14-9496     [1, 2, 3, 4, 6, 7, 8, 0]
3   15-1031     [1, 2, 3, 4, 6, 7, 8, 0]
4   15-1039     [1, 3, 4, 5, 6, 7, 8, 0]
5   15-1111        [1, 3, 4, 6, 7, 8, 0]
6    15-118     [1, 2, 3, 4, 6, 7, 8, 0]
7   15-1189           [1, 2, 3, 7, 8, 0]
8   15-1191     [1, 2, 3, 4, 6, 7, 8, 0]
9   15-1194     [1, 2, 3, 4, 6, 7, 8, 0]
10  15-1204     [1, 2, 3, 4, 6, 7, 8, 0]
11  15-1248     [1, 2, 3, 4, 6, 7, 8, 0]
12  15-1251     [1, 2, 3, 4, 6, 7, 8, 0]
13  15-1256     [1, 2, 3, 4, 6, 7, 8, 0]
14  15-1262     [1, 2, 3, 4, 6, 7, 8, 0]
15  15-1293     [1, 2, 3, 4, 6, 7, 8, 0]
16  15-1358           [1, 2, 3, 4, 7, 0]
17  15-1391     [1, 2, 3, 4, 6, 7, 8, 0]
18  15-1406     [1, 2, 3, 4, 6, 7, 8, 0]
19  15-1498     [1, 2, 3, 4, 6, 7, 8, 0]
20  15-1500     [1, 2, 3, 4, 6, 7, 8, 0]
21  15-1503     [1, 2, 3, 4, 6, 7, 8, 0]
22   15-214     [1, 2, 3, 4, 6, 7, 8, 0]
23   15-423     [1, 2, 3, 4, 6, 7, 8, 0]
24   15-457     [1, 2, 3, 4, 6, 7, 8, 0]
25   15-497     [1, 2, 3, 4, 6, 7, 8, 0]
26   15-513     [1, 2, 3, 4, 6, 7, 8, 0]
27   15-537        [1, 2, 4, 6, 7, 8, 0]
28   15-577  [1, 2, 3, 4, 5, 6, 7, 8, 0]
29  15-5991     [1, 2, 3, 4, 6, 7, 8, 0]
..      ...                          ...
34  15-7250        [1, 3, 4, 6, 7, 8, 0]
35   15-777     [1, 2, 3, 4, 6, 7, 8, 0]
36   15-797     [1, 2, 3, 4, 6, 7, 8, 0]
37  15-8049     [1, 2, 3, 4, 6, 7, 8, 0]
38   15-827     [1, 2, 3, 4, 6, 7, 8, 0]
39  15-8544        [1, 2, 3, 4, 7, 8, 0]
40   15-866     [1, 2, 3, 4, 6, 7, 8, 0]
41  15-9260     [1, 2, 3, 4, 6, 7, 8, 0]
42   15-927        [1, 2, 3, 4, 6, 8, 0]
43   16-142     [1, 2, 3, 4, 6, 7, 8, 0]
44   16-149     [1, 2, 3, 4, 6, 7, 8, 0]
45   16-240  [1, 2, 3, 4, 5, 6, 7, 8, 0]
46   16-254     [1, 2, 3, 4, 6, 7, 8, 0]
47   16-309  [1, 2, 3, 4, 5, 6, 7, 8, 0]
48    16-32     [1, 2, 3, 4, 6, 7, 8, 0]
49   16-327     [1, 2, 3, 4, 6, 7, 8, 0]
50   16-341        [1, 3, 4, 6, 7, 8, 0]
51   16-348     [1, 2, 3, 4, 6, 7, 8, 0]
52   16-349        [1, 2, 3, 4, 6, 8, 0]
53   16-369     [1, 2, 3, 4, 6, 7, 8, 0]
54   16-373  [1, 2, 3, 4, 5, 6, 7, 8, 0]
55   16-399  [1, 2, 3, 4, 5, 6, 7, 8, 0]
56   16-405  [1, 2, 3, 4, 5, 6, 7, 8, 0]
57   16-466  [1, 2, 3, 4, 5, 6, 7, 8, 0]
58   16-529  [1, 2, 3, 4, 5, 6, 7, 8, 0]
59  16-5294  [1, 2, 3, 4, 5, 6, 7, 8, 0]
60    16-54     [1, 2, 3, 4, 6, 7, 8, 0]
61   16-605  [1, 2, 3, 4, 5, 6, 7, 8, 0]
62  16-6219  [1, 2, 3, 4, 5, 6, 7, 8, 0]
63    16-74     [1, 2, 3, 4, 6, 7, 8, 0]

[64 rows x 2 columns]

In [332]:
a['color'] = "['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86','0BD488','000000','7E3B0D']"

In [340]:
a

docket                   speak_list  \
0   14-1055        [1, 3, 4, 6, 7, 8, 0]   
1   14-1538     [1, 2, 3, 4, 6, 7, 8, 0]   
2   14-9496     [1, 2, 3, 4, 6, 7, 8, 0]   
3   15-1031     [1, 2, 3, 4, 6, 7, 8, 0]   
4   15-1039     [1, 3, 4, 5, 6, 7, 8, 0]   
5   15-1111        [1, 3, 4, 6, 7, 8, 0]   
6    15-118     [1, 2, 3, 4, 6, 7, 8, 0]   
7   15-1189           [1, 2, 3, 7, 8, 0]   
8   15-1191     [1, 2, 3, 4, 6, 7, 8, 0]   
9   15-1194     [1, 2, 3, 4, 6, 7, 8, 0]   
10  15-1204     [1, 2, 3, 4, 6, 7, 8, 0]   
11  15-1248     [1, 2, 3, 4, 6, 7, 8, 0]   
12  15-1251     [1, 2, 3, 4, 6, 7, 8, 0]   
13  15-1256     [1, 2, 3, 4, 6, 7, 8, 0]   
14  15-1262     [1, 2, 3, 4, 6, 7, 8, 0]   
15  15-1293     [1, 2, 3, 4, 6, 7, 8, 0]   
16  15-1358           [1, 2, 3, 4, 7, 0]   
17  15-1391     [1, 2, 3, 4, 6, 7, 8, 0]   
18  15-1406     [1, 2, 3, 4, 6, 7, 8, 0]   
19  15-1498     [1, 2, 3, 4, 6, 7, 8, 0]   
20  15-1500     [1, 2, 3, 4, 6, 7, 8, 0]   
21  15-1503     [1, 2, 3, 4, 6, 7, 8, 0]   
22   15-214     [1, 2, 3, 4, 6, 7, 8, 0]   
23   15-423     [1, 2, 3, 4, 6, 7, 8, 0]   
24   15-457     [1, 2, 3, 4, 6, 7, 8, 0]   
25   15-497     [1, 2, 3, 4, 6, 7, 8, 0]   
26   15-513     [1, 2, 3, 4, 6, 7, 8, 0]   
27   15-537        [1, 2, 4, 6, 7, 8, 0]   
28   15-577  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
29  15-5991     [1, 2, 3, 4, 6, 7, 8, 0]   
..      ...                          ...   
34  15-7250        [1, 3, 4, 6, 7, 8, 0]   
35   15-777     [1, 2, 3, 4, 6, 7, 8, 0]   
36   15-797     [1, 2, 3, 4, 6, 7, 8, 0]   
37  15-8049     [1, 2, 3, 4, 6, 7, 8, 0]   
38   15-827     [1, 2, 3, 4, 6, 7, 8, 0]   
39  15-8544        [1, 2, 3, 4, 7, 8, 0]   
40   15-866     [1, 2, 3, 4, 6, 7, 8, 0]   
41  15-9260     [1, 2, 3, 4, 6, 7, 8, 0]   
42   15-927        [1, 2, 3, 4, 6, 8, 0]   
43   16-142     [1, 2, 3, 4, 6, 7, 8, 0]   
44   16-149     [1, 2, 3, 4, 6, 7, 8, 0]   
45   16-240  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
46   16-254     [1, 2, 3, 4, 6, 7, 8, 0]   
47   16-309  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
48    16-32     [1, 2, 3, 4, 6, 7, 8, 0]   
49   16-327     [1, 2, 3, 4, 6, 7, 8, 0]   
50   16-341        [1, 3, 4, 6, 7, 8, 0]   
51   16-348     [1, 2, 3, 4, 6, 7, 8, 0]   
52   16-349        [1, 2, 3, 4, 6, 8, 0]   
53   16-369     [1, 2, 3, 4, 6, 7, 8, 0]   
54   16-373  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
55   16-399  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
56   16-405  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
57   16-466  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
58   16-529  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
59  16-5294  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
60    16-54     [1, 2, 3, 4, 6, 7, 8, 0]   
61   16-605  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
62  16-6219  [1, 2, 3, 4, 5, 6, 7, 8, 0]   
63    16-74     [1, 2, 3, 4, 6, 7, 8, 0]   

                                                color  
0   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
1   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
2   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
3   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
4   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
5   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
6   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
7   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
8   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
9   ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
10  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
11  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
12  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
13  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
14  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
15  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
16  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
17  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
18  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
19  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
20  ['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86'...  
21  ['CE21C3', '0BB2E8','37CB00','8A04D8','D

# Merge all the tables

In [482]:
f_df = f_df.merge(output, left_on = 'docket', right_on = 'docket')
f_df = f_df.merge(a, left_on = 'docket', right_on = 'docket')
f_df = f_df.merge(info_df2, left_on = 'docket', right_on = 'docket')
f_df

caseName   docket      fileName  \
0               Perry v. Merit Systems Protection Bd.   16-399   16-399_3f14   
1              Town of Chester v. Laroe Estates, Inc.   16-605   16-605_2dp3   
2   California Public Employees' Retirement System...   16-373   16-373_4e46   
3                                       Kokesh v. SEC   16-529   16-529_21p3   
4               Henson v. Santander Consumer USA Inc.   16-349   16-349_e29g   
5   Trinity Lutheran Church of Columbia, Inc. v. C...   15-577   15-577_l64n   
6                             Weaver v. Massachusetts   16-240   16-240_nkp1   
7                                  McWilliams v. Dunn  16-5294  16-5294_g314   
8                                     Davila v. Davis  16-6219  16-6219_7mio   
9   Bristol-Myers Squibb Co. v. Superior Court of ...   16-466   16-466_4g15   
10                             BNSF R. Co. v. Tyrrell   16-405   16-405_9olb   
11                          Sandoz Inc. v. Amgen Inc.  15-1039  15-1039_bqm1   
12                         Maslenjak v. United States   16-309   16-309_b97c   
13                                  Murr v. Wisconsin   15-214   15-214_l6hn   
14                                   Howell v. Howell  15-1031  15-1031_6647   
15                           Microsoft Corp. v. Baker   15-457   15-457_gfbh   
16  Impressions Products, Inc. v. Lexmark Int'l, Inc.  15-1189  15-1189_6468   
17                    County of Los Angeles v. Mendez   16-369   16-369_8nka   
18                        Water Splash, Inc. v. Menon   16-254   16-254_7lio   
19          Advocate Health Care Network v. Stapleton    16-74    16-74_p8k0   
20   TC Heartland LLC v. Kraft Foods Group Brands LLC   16-341   16-341_8njq   
21                               Lee v. United States   16-327   16-327_d18e   
22                            Turner v. United States  15-1503  15-1503_3f14   
23                         Honeycutt v. United States   16-142   16-142_4gc5   
24                                  Hernandez v. Mesa   15-118   15-118_3e04   
25                                 McLane Co. v. EEOC  15-1248  15-1248_2dq3   
26              Kindred Nursing Centers Ltd. v. Clark    16-32    16-32_mlho   
27                       Packingham v. North Carolina  15-1194  15-1194_0861   
28                      Esquivel-Quintana v. Sessions    16-54    16-54_7l48   
29                              Dean v. United States  15-9260  15-9260_bq7c   
..                                                ...      ...           ...   
32                                    Lewis v. Clarke  15-1500  15-1500_5g68   
33            Expressions Hair Design v. Schneiderman  15-1391  15-1391_5315   
34               Goodyear Tire & Rubber Co. v. Haeger  15-1406  15-1406_d1of   
35      Endrew F. v. Douglas County School Dist. RE-1   15-827   15-827_gfbh   
36                    Midland Funding, LLC v. Johnson   16-348   16-348_2cp3   
37                                         Lee v. Tam  15-1293  15-1293_o7jp   
38                                   Ziglar v. Abbasi  15-1358  15-1358_7648   
39                           Beckles v. United States  15-8544  15-8544_c1o2   
40                                     Moore v. Texas   15-797   15-797_f2q3   
41    Bethune-Hill v. Virginia State Bd. of Elections   15-680   15-680_n648   
42                                  McCrory v. Harris  15-1262  15-1262_l537   
43           Life Technologies Corp. v. Promega Corp.  14-1538  14-1538_j4ek   
44                   Czyzewski v. Jevic Holding Corp.   15-649   15-649_l5gm   
45                  Fry v. Napoleon Community Schools   15-497   15-497_4g15   
46     Star Athletica, L.L.C. v. Varsity Brands, Inc.   15-866   15-866_j426   
47  State Farm Fire & Casualty Co. v. United State...   15-513   15-513_k5fm   
48  SCA Hygiene Products Aktiebolag v. First Quali...   15-927   15-927_6j37   
49  Bolivarian Republic of Venezuela v. Helmerich ...   15-423   15-423_pnk0   
50                           NLRB v. SW Gener

# Turn the longtitude and latitude into geopandas points

In [484]:
f_df['lat'] = f_df['location'].apply(lambda x: x.split(',')[0])
f_df['lon'] = f_df['location'].apply(lambda x: x.split(',')[1])
f_df['lat'] = f_df['lat'].astype(float)
f_df['lon'] = f_df['lon'].astype(float)

In [485]:
ff_df = f_df[['caseName', 'docket', 'granted', 'argued', 'decided', 'lat', 'lon', 'summary', 'vote', 's_conclusion','f_speaker_line', 'descriptions', 'conclusion', 'article', 'speak_list', 'color']]
ff_df

caseName   docket       granted  \
0               Perry v. Merit Systems Protection Bd.   16-399  Jan 13, 2017   
1              Town of Chester v. Laroe Estates, Inc.   16-605  Jan 13, 2017   
2   California Public Employees' Retirement System...   16-373  Jan 13, 2017   
3                                       Kokesh v. SEC   16-529  Jan 13, 2017   
4               Henson v. Santander Consumer USA Inc.   16-349  Jan 13, 2017   
5   Trinity Lutheran Church of Columbia, Inc. v. C...   15-577  Jan 15, 2016   
6                             Weaver v. Massachusetts   16-240  Jan 13, 2017   
7                                  McWilliams v. Dunn  16-5294  Jan 13, 2017   
8                                     Davila v. Davis  16-6219  Jan 13, 2017   
9   Bristol-Myers Squibb Co. v. Superior Court of ...   16-466  Jan 19, 2017   
10                             BNSF R. Co. v. Tyrrell   16-405  Jan 13, 2017   
11                          Sandoz Inc. v. Amgen Inc.  15-1039  Jan 13, 2017   
12                         Maslenjak v. United States   16-309  Jan 13, 2017   
13                                  Murr v. Wisconsin   15-214  Jan 15, 2016   
14                                   Howell v. Howell  15-1031   Dec 2, 2016   
15                           Microsoft Corp. v. Baker   15-457  Jan 15, 2016   
16  Impressions Products, Inc. v. Lexmark Int'l, Inc.  15-1189   Dec 2, 2016   
17                    County of Los Angeles v. Mendez   16-369   Dec 2, 2016   
18                        Water Splash, Inc. v. Menon   16-254  Feb 12, 2016   
19          Advocate Health Care Network v. Stapleton    16-74   Dec 2, 2016   
20   TC Heartland LLC v. Kraft Foods Group Brands LLC   16-341  Dec 14, 2016   
21                               Lee v. United States   16-327  Dec 14, 2016   
22                            Turner v. United States  15-1503  Dec 14, 2016   
23                         Honeycutt v. United States   16-142   Dec 9, 2016   
24                                  Hernandez v. Mesa   15-118  Oct 11, 2016   
25                                 McLane Co. v. EEOC  15-1248  Sep 29, 2016   
26              Kindred Nursing Centers Ltd. v. Clark    16-32  Oct 28, 2016   
27                       Packingham v. North Carolina  15-1194  Oct 28, 2016   
28                      Esquivel-Quintana v. Sessions    16-54  Oct 28, 2016   
29                              Dean v. United States  15-9260  Oct 28, 2016   
..                                                ...      ...           ...   
32                                    Lewis v. Clarke  15-1500  Sep 29, 2016   
33            Expressions Hair Design v. Schneiderman  15-1391  Sep 29, 2016   
34               Goodyear Tire & Rubber Co. v. Haeger  15-1406  Sep 29, 2016   
35      Endrew F. v. Douglas County School Dist. RE-1   15-827  Sep 29, 2016   
36                    Midland Funding, LLC v. Johnson   16-348  Oct 11, 2016   
37                                         Lee v. Tam  15-1293  Sep 29, 2016   
38                                   Ziglar v. Abbasi  15-1358  Oct 11, 2016   
39                           Beckles v. United States  15-8544  Jun 27, 2016   
40                                     Moore v. Texas   15-797   Jun 6, 2016   
41    Bethune-Hill v. Virginia State Bd. of Elections   15-680   Jun 6, 2016   
42                                  McCrory v. Harris  15-1262  Jun 27, 2016   
43           Life Technologies Corp. v. Promega Corp.  14-1538  Jun 27, 2016   
44                   Czyzewski v. Jevic Holding Corp.   15-649  Jun 28, 2016   
45                  Fry v. Napoleon Community Schools   15-497  Jun 28, 2016   
46     Star Athletica, L.L.C. v. Varsity Brands, Inc.   15-866   May 2, 2016   
47  State Farm Fire & Casualty Co. v. United State...   15-513  May 31, 2016   
48  SCA Hygiene Products Aktiebolag v. First Quali...   15-927   May 2, 2016   
49  Bolivarian Republic of Venezuela v. Helmerich ...   15-423  Jun 28, 2016   
50                           NLRB v. SW Gener

In [486]:
gf_df = gpd.GeoDataFrame(ff_df)

In [487]:
gf_df

caseName   docket       granted  \
0               Perry v. Merit Systems Protection Bd.   16-399  Jan 13, 2017   
1              Town of Chester v. Laroe Estates, Inc.   16-605  Jan 13, 2017   
2   California Public Employees' Retirement System...   16-373  Jan 13, 2017   
3                                       Kokesh v. SEC   16-529  Jan 13, 2017   
4               Henson v. Santander Consumer USA Inc.   16-349  Jan 13, 2017   
5   Trinity Lutheran Church of Columbia, Inc. v. C...   15-577  Jan 15, 2016   
6                             Weaver v. Massachusetts   16-240  Jan 13, 2017   
7                                  McWilliams v. Dunn  16-5294  Jan 13, 2017   
8                                     Davila v. Davis  16-6219  Jan 13, 2017   
9   Bristol-Myers Squibb Co. v. Superior Court of ...   16-466  Jan 19, 2017   
10                             BNSF R. Co. v. Tyrrell   16-405  Jan 13, 2017   
11                          Sandoz Inc. v. Amgen Inc.  15-1039  Jan 13, 2017   
12                         Maslenjak v. United States   16-309  Jan 13, 2017   
13                                  Murr v. Wisconsin   15-214  Jan 15, 2016   
14                                   Howell v. Howell  15-1031   Dec 2, 2016   
15                           Microsoft Corp. v. Baker   15-457  Jan 15, 2016   
16  Impressions Products, Inc. v. Lexmark Int'l, Inc.  15-1189   Dec 2, 2016   
17                    County of Los Angeles v. Mendez   16-369   Dec 2, 2016   
18                        Water Splash, Inc. v. Menon   16-254  Feb 12, 2016   
19          Advocate Health Care Network v. Stapleton    16-74   Dec 2, 2016   
20   TC Heartland LLC v. Kraft Foods Group Brands LLC   16-341  Dec 14, 2016   
21                               Lee v. United States   16-327  Dec 14, 2016   
22                            Turner v. United States  15-1503  Dec 14, 2016   
23                         Honeycutt v. United States   16-142   Dec 9, 2016   
24                                  Hernandez v. Mesa   15-118  Oct 11, 2016   
25                                 McLane Co. v. EEOC  15-1248  Sep 29, 2016   
26              Kindred Nursing Centers Ltd. v. Clark    16-32  Oct 28, 2016   
27                       Packingham v. North Carolina  15-1194  Oct 28, 2016   
28                      Esquivel-Quintana v. Sessions    16-54  Oct 28, 2016   
29                              Dean v. United States  15-9260  Oct 28, 2016   
..                                                ...      ...           ...   
32                                    Lewis v. Clarke  15-1500  Sep 29, 2016   
33            Expressions Hair Design v. Schneiderman  15-1391  Sep 29, 2016   
34               Goodyear Tire & Rubber Co. v. Haeger  15-1406  Sep 29, 2016   
35      Endrew F. v. Douglas County School Dist. RE-1   15-827  Sep 29, 2016   
36                    Midland Funding, LLC v. Johnson   16-348  Oct 11, 2016   
37                                         Lee v. Tam  15-1293  Sep 29, 2016   
38                                   Ziglar v. Abbasi  15-1358  Oct 11, 2016   
39                           Beckles v. United States  15-8544  Jun 27, 2016   
40                                     Moore v. Texas   15-797   Jun 6, 2016   
41    Bethune-Hill v. Virginia State Bd. of Elections   15-680   Jun 6, 2016   
42                                  McCrory v. Harris  15-1262  Jun 27, 2016   
43           Life Technologies Corp. v. Promega Corp.  14-1538  Jun 27, 2016   
44                   Czyzewski v. Jevic Holding Corp.   15-649  Jun 28, 2016   
45                  Fry v. Napoleon Community Schools   15-497  Jun 28, 2016   
46     Star Athletica, L.L.C. v. Varsity Brands, Inc.   15-866   May 2, 2016   
47  State Farm Fire & Casualty Co. v. United State...   15-513  May 31, 2016   
48  SCA Hygiene Products Aktiebolag v. First Quali...   15-927   May 2, 2016   
49  Bolivarian Republic of Venezuela v. Helmerich ...   15-423  Jun 28, 2016   
50                           NLRB v. SW Gener

In [488]:
gf_df['geometry'] = gf_df.apply(lambda row: Point(row.lon, row.lat), axis=1)
gf_df['speak_list'] = gf_df.speak_list.astype(str)

In [489]:
gf_df.to_file('case500.json', driver='GeoJSON')